In [1]:
'''Generating grassmannians tableaux'''
'''this needs to be run on SageMath 9.0 or higher on a Linux system'''
#Import libraries
import sys
import numpy as np
import itertools  
from operator import add
from typing import List
from sage.combinat.shuffle import ShuffleProduct
from sage.parallel.multiprocessing_sage import Pool
import multiprocessing  
import random   
import os
import subprocess
import functools 
import multiprocessing  
import operator
 
####################################################################
#Define relevant functions

def ChTUsingKLPolynomialInSageHalfUseCProgramSLk(k,l,w,w0,W,T1): 
    
    fp1="pairsOfWords.txt" 
    
    r=[] 
    t1=ShiftByA(l.tuple(),0)
    t2=SymmetricGroupActionOnTableauxComputeChSLk('Y',k,t1,T1)
    #print('t1,t2',[t1,t2])
    if t2!=[]:
        p1=(l*w0).tuple()
        p2=(w*w0).tuple()
        #print(p1,p2)
        p1=ShiftByA(p1,-1)
        p2=ShiftByA(p2,-1)
        

        t4=(-1)^((l*w).length())
        r.append([t4, [p1,p2], t2])
    return r

def SymmetricGroupActionOnListSi(i,L): #  w=s_i,  L=[a1,a2,...,an]
    r1=[]
    for j in L:
        r1.append(j)
    
    t1=r1[i-1]
    r1[i-1]=r1[i]
    r1[i]=t1 
     
    r=r1
        
    return r

def FundamentalTableauToWeightSLk(a1,k): # SLk, T_{l,a}, length l, last entry a, T_{l,a}=T^{l+k-a,l} \sim Y_{l+k-a,k-a-l}, weight of T_{l,a} is omiga_(l+k-a)
 
    r=[]
    rank=k-1
    n=rank
    for i in [1..n]:
        r=append(r,0)
        
    a=sorted(a1)
    l=len(a)
    t1=a[len(a)-1]
    t2=l+k-t1
    
    r[t2-1]=1
     
    #print(r)

    return r

def AddVlueOfTwoLists(a,b):
    r=a
    for i in [0..len(a)-1]:
        r[i]=a[i]+b[i]
    return r

def WeightOfTableau2SLk(a,k): # a[i] are columns of the tableau a, SLk, k=rank+1
    r1=TableauToFundamentalTableauxSLk('Y', a)
    rank=k-1
    n=rank
    
    r=[]
    for i in [1..n]:
        r=append(r,0)
    
    #print(r)
    
    for i in r1:
        t1=FundamentalTableauToWeightSLk(i,k)
        #print(t1)
        r=AddVlueOfTwoLists(r,t1) 
        
    return r

def compareWeightsTableaux2SLk(P1,P2,k): # SLk
    typ='A'
    rank=k-1
    if P1!=[]:
        t1=WeightOfTableau2SLk(P1,k)
    else:
        t1=[]
        for i in [1..rank]:
            t1.append(0)

    if P2!=[]:
        t2=WeightOfTableau2SLk(P2,k)
    else:
        t2=[]
        for i in [1..rank]:
            t2.append(0)

    #print(t1,t2)

    r=compareWeights2(t1,t2,typ, rank)

    return r

def HighestWeightTableauInPolynomialOfTableauxSLk(l,k): # SLk
    rank=k-1
    typ='A'
    if l==[]:
        r=[]
    else:
        r=l[0]
        for i in [1..len(l)-1]:
            if compareWeightsTableaux2SLk(r[1], l[i][1],rank)==-1:
                
                #print('r[1]',TableauToModuleHasY(Y,r[1]))
                #print('l[i][1]',TableauToModuleHasY(Y,l[i][1]))
                r=l[i]
                
    return r

def RemoveTrivialTableauxInStandardMonomialSLk(M,k): # M = [[...],[...]], each [...] is a Plucker coordinate
    r=[]
    r1=PluckersWhichAreOneSLk(k)
    for i in M:
        if IsInList(i, r1)==0:
            r.append(i)
    r=sorted(r)
    return r

def RemoveTrivialTableauxInStandardMonomialPolynomialSLk(P,k): # each term in P is [coeff, M], M = [[...],[...]], each [...] is a Plucker coordinate
    r=[]
    for i in P:
        r.append( [i[0], RemoveTrivialTableauxInStandardMonomialSLk(i[1],k)] )
    return r

def MultiplyTwoPolynomialsLists2(a,b):

    r=[]
    for i in a:
        for j in b:
            temp1=i[0]*j[0]
            temp2=i[1]+j[1]
            r.append([temp1,temp2])
    return r

def MultiplyTwoPolynomialsLists2Many(e): # e is a list of polynomials in terms lists
    if e==[]:
        r=[]
    elif len(e)==1:
        r=e[0]
    elif len(e)>=2:
        r=MultiplyTwoPolynomialsLists2(e[0],e[1])
        for i in [2..len(e)-1]:
            r=MultiplyTwoPolynomialsLists2(r,e[i])
    
    return r

def Multiplyq2(l,q):
 
    r=[]
    for i in l:
        temp=i
        temp[0]=simplify(q*temp[0])
        r.append(temp)
 
    return r

def Multiplyq2NoChangeValue(l,q):
 
    r=[]
    for i in l:
        t1=[simplify(q*i[0]), i[1]]
        r.append(t1)
 
    return r

def IsCompatiblePairTableauxInMatrixSLkUsingChT(k,TT1,TT2): # TT1[i] are columns
    typ='A' 
    rank1=k-1
    t1=TT1
    t2=TT2
    d=[t1,t2]

    e=[] 
    e1=[]
    e2=[]
    for T1 in d:
        tt3=ChTUsingKLPolynomialInSageSLk(T1,k)
        e.append(tt3)

    f4=MultiplyTwoPolynomialsLists2Many(e) 
    f4=RemoveTrivialTableauxInStandardMonomialPolynomialSLk(f4,k)
    f4=combineLikeTerms3(f4)

    while f4!=[]:
        g1=HighestWeightTableauInPolynomialOfTableauxSLk(f4,k)
        print([g1[0], RemoveTrivialTableauSLk(g1[1],k)])
    #     t1=[g1[0], TableauToMultiSegments(Y,g1[1]), TableauToModuleHasY(Y,g1[1]), TableauToModuleInMatrixPAdicConventionHasY(Y,TableauToMatrix(g1[1]))]
    #     print(t1) 
        e1.append([g1[0], RemoveTrivialTableauSLk(g1[1],k)])
    #     e2.append(sorted(t1[1]))
        T1=g1[1]
        if T1==[]: # trivial module, corresponding to \mathbb{C}
            c2=[[1, []]]
        else:
            c2=ChTUsingKLPolynomialInSageSLk(T1,k)
        c2=Multiplyq2NoChangeValue(c2, g1[0])
        f4=PolynomialE1SubtractE2EiArePolynomialsInStandardMonomialsInTermsOfTableauxSLk(f4,c2,k)
        
    if len(e1)==1:
        r=1
    elif len(e1)>1:
        r=0
    
    return r

def PolynomialE1SubtractE2EiArePolynomialsInStandardMonomialsInTermsOfTableauxSLk(e1,e2,k): # e1-e2
    e3=e1+Multiplyq2NoChangeValue(e2,-1)
    e4=RemoveTrivialTableauxInStandardMonomialPolynomialSLk(e3,k)
    r=combineLikeTerms3(e4)
    
    return r

def StablizerOfTuple(A): # A is a list, weakly increasing
    k = len(A)
    r=[]
    for i in [1..k-1]:
        t1=SymmetricGroupActionOnListSi(i,A)
        if t1==A:
            r.append(i)
    return r

def LongestPermWInSBWInverseSA(W,A,B): # longest word in S_j w^{-1} S_i, is it longest word in S_{lambda} w S_{mu}?
    t1=StablizerOfTuple(A)
    t2=StablizerOfTuple(sorted(B))
    s=W.simple_reflections()
    #print(t1,t2)
    t3=[]
    for i in t1:
        t3.append(s[i])
    t4=[]
    for i in t2:
        t4.append(s[i])
    t5=find_permutation2(B,sorted(B))
    w=W.one()
    for i in t5:
        w=w*s[i]
    #print(t4,w,t3)
    r2=LongestPermInDoubleCosetWeylGroup(W,t4,w,t3)
    r=r2
    
    return r

def myprod(g1,w,u2):
    return max([t*w*u2 for t in g1], key=lambda t: t.length())

def LongestPermInDoubleCosetWeylGroup(W,S1,w,S2): # W is a Weyl group, typ='A', rank=4, W=WeylGroup(typ+str(rank), prefix = 's')
    g1=W.subgroup(S1)
    g2=W.subgroup(S2)
    winner = W.one()
    N_CPU = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=N_CPU) as pool:
        winner = max( pool.imap_unordered(functools.partial(myprod,g1,w), g2), key=lambda t: t.length() )      
#     for u1 in g1:
#         for u2 in g2:
#             t1=u1*w*u2
#             if t1.length()>winner.length():
#                 winner=t1
    r=winner
    
    return r

def find_permutation2(L1, L2):
    perm = Word(L1).standard_permutation() / Word(L2).standard_permutation()
    assert [L2[i-1] for i in perm] == L1
    r=perm.reduced_word()
    return r

def ChTUsingKLPolynomialInSageSLk(T1, k): # T1 is a tableau in the case of C[SLk/N] or C[N], T1[i]'s are columns of the tableau
    
    fp1="pairsOfWords.txt"
    fp3="HL8results18.txt"
    fp5="tableaux.txt"
    
    t2=TableauToFundamentalTableauxSLk('Y',T1)
    #print(t2)
    t3=TableauToListsIJSLk('Y',t2)
    #print(t3)
    A=t3[0]
    B=t3[1] 
    kk=len(A)
    
    print(kk)
    
    typ='A'
    rank=k

    W = SymmetricGroup(kk)
    
    w=LongestPermWInSBWInverseSA(W,A,B)
    
    print('w', w)

    w0=W.long_element()

    #print(w0.reduced_word()) 
    
#     F1 = open(fp1,'w')
#     F1.write(str(''))
#     F1.close()
    
#     F1 = open(fp5,'w')
#     F1.write(str(''))
#     F1.close()
    
    N_CPU = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=N_CPU) as pool:
        bb = pool.starmap(ChTUsingKLPolynomialInSageHalfUseCProgramSLk,[(k,i,w,w0,W,T1) for i in iter(W)])
                
    r1=[]
    for i in bb:
        r1=r1+i
        
    #print('r1',r1)
    
    r2=[]
    r3=[]
    r4=[]
    for t1 in r1:
        r2.append(t1[0])
        p1=t1[1][0]
        p2=t1[1][1]
        r3.append([p1,p2])
        r4.append(t1[2])
    
    F1 = open(fp1,'w')
    for i in r3:
        p1=i[0]
        p2=i[1]
        #print(p1,p2)
        for j in p1:
            F1.write(chr(j+48))
        F1.write(str(' '))
        for j in p2:
            F1.write(chr(j+48))
        F1.write(str('\n'))
    F1.close()
    
    cmd = "./a.out "+str(kk)+" pairsOfWords.txt | awk '{print $NF}' > HL8results18.txt"
    os.system(cmd)
    
    with open(fp3, 'r') as fp:
        L = [sage_eval(line) for line in fp.readlines() if line.strip()]
        
    #print('L',L)
        
    t1=[]
    for i in L:
        if type(i)==Integer:
            t1.append([i])
        else:
            t2=[]
            for j in i:
                t2.append(j)
            t1.append(t2)
    PL=ListToPolynomialInQList(t1,1)
    
    #print('t1',t1)
    #print('PL',PL)
    
    r=[]
    for i in range(len(r2)):
        if PL[i]!=0:
            t1=r2[i]*PL[i]
            r.append([t1, r4[i]])
            
    r=combineLikeTerms3(r)
    
    return r

def UnionOfTwoTableauxSLk(a,b): # a[i], b[i] are columns
    if a==[]:
        r=b
    elif b==[]:
        r=a
    else:
        r1=TransposeTableauSLk(a)
        r2=TransposeTableauSLk(b)
        
        m=max(len(r1), len(r2))
        #print('m,len(r1),len(r2)',m,len(r1),len(r2))
        r3=[]
        for i in [0..m-1]:
            #print(i,len(r1),len(r2))
            if i<len(r1) and i<len(r2):
                t2=r1[i]+r2[i]
                #print('t2',t2)
                t2=sorted(t2)
            elif i>=len(r1) and i<len(r2):
                    t2=sorted(r2[i])
            elif i<len(r1) and i>=len(r2):
                    t2=sorted(r1[i])
            r3.append(t2)
        #print('r3',r3)
        r=TransposeTableauSLk(r3)
    return r

def UnionOfTableauxManySLk(a):
 
    r=0

    if a!=[]:
        r=a[0]
        for i in [1..len(a)-1]:
            r=UnionOfTwoTableauxSLk(r, a[i])
         
    return r

def IsWeaklySeparatedSLk(I1,J1): # I1,J1 have the same size
 
    r=0
    
    r1=sorted(SetDifference(I1,J1))
    r2=sorted(SetDifference(J1,I1))
    
    if r1==[] or r2==[]:
        r=1
        return r
    
    else:
    
        if len(I1)<=len(J1):
            m1=min(r2)
            m2=max(r2)

            t1=[]
            t2=[]
            for i in r1:
                if i<m1:
                    t1.append(i)
                elif i>m2:
                    t2.append(i)
            if r1==sorted(t1+t2):
                r=1
                return r

        elif len(I1)>len(J1):
            m1=min(r1)
            m2=max(r1)

            t1=[]
            t2=[]
            for i in r2:
                if i<m1:
                    t1.append(i)
                elif i>m2:
                    t2.append(i)
            if r2==sorted(t1+t2):
                r=1
                return r
        
    return r


def SetDifference(A,B): # A-B
    r=[]
    for i in A:
        if IsInList(i,B)==0:
            r.append(i)
    return r

def CartesianProductOfListsInAList(l): # l=[l1,l2,...], l1 is a set written in terms of a list
 
    r=[]
    for i in itertools.product(*l):
        r.append(i)

    return r 

def IsCompatiblePairTableauxInMatrixSLk(k,T1, T2): # T1[i] are columns
 

    r=0

    if IsLCTTPrimeNewSLk(k,T1, T2)==1 and IsLCTTPrimeNewSLk(k,T2, T1)==1:
        r=1 

    return r 

def IsLCTTPrimeNewSLk(k,T1, T2):
 
    m1=TableauToMultisegmentSLk('Y',k,T1)
    m2=TableauToMultisegmentSLk('Y',k,T2)

    r=0

    t1=IsLCkMMPrimeNew(k,m1,m2)
    t2=IsLCkMMPrimeNew(k,m2,m1)

    if t1==1 and t2==1:
        r=1
        return r
    
    return r

def TableauToMultisegmentSLk(Y,k,l): # l is a tableau T, l[i] is column of i of T, SLk
    
    t1=TableauToModuleSLk(Y,k,l)
 
    r=ModuleToMultiSegment(Y,t1)

    return r

def ModuleToMultiSegment(Y,l): # l is a list, l[i] is the index of some Y[i,j] 
     
    r=TranslateModuleToMultiSegmentsMonomial(Y,l)

    return r

def TranslateModuleToMultiSegmentsMonomial(Y,l): # l is a list, l[i] is the index of some Y[i,j]
 
    r=[]
    for j in l: 
        r.append(TranslateQuantumAffineToLadders(j))
        
    
    r=SortBySecondTerms(r)

    return r

def TranslateQuantumAffineToLadders(l):
    i=l[0]
    s=l[1] 
    r=[(s-i)/2+1/2, (s+i)/2-1/2]

    r=[r[0]+1/2, r[1]+1/2]

    return r

def SortBySecondTerms(l):

    r=[]

    if l!=[]: 

        t1=[]
        for i in l:
            t1=append(t1, i[1])

        t1=sorted(t1) 
        t2=sorted(l)

        r=[] 
        for i in reversed([min(t1)..max(t1)]):
            for j in t2:
                if j[1]==i:
                    r=append(r, j)
    return r

def IsLCkMMPrimeNew(k,m1,m2): # LC_k(m, m')

    r=0

    a=TheSetXkMMPrime(k,m1,m2)
    b=TheSetYkMMPrime(k,m1,m2)
 
    if len(a)>len(b):
        r=0
    elif len(a)==0:
        r=1
    else:

        r1=[]
        for p1 in a:
            t1=[]
            for p2 in b:
                i=p1[0]-1
                j=p1[1]-1
                i1=p2[0]-1
                j1=p2[1]-1
                if ( i==i1 and IsSegementAkPrecB(k, m2[j1], m2[j])==1 ) or ( j==j1 and IsSegementAkPrecB(k, m1[i], m1[i1])==1 ):
                    t1=append(t1, p2)
            
            if t1==[]: # can speed up the computations
                r=0
                return r
                break
            r1=append(r1, t1) 

        r3=CartesianProductOfListsInAList(r1) 

        for i in r3:
            t1=removeDuplicates(i) 
            if len(t1)==len(a): 
                r=1
                return r
                break
                

    return r 

def TheSetXkMMPrime(k,m1,m2): # m1, m2 are multisegments

    r=[]
    for i in [0..len(m1)-1]:
        for j in [0..len(m2)-1]:
            if IsSegementAkPrecB(k, m1[i], m2[j] )==1:
                #r.append([i,j]) 
                r.append([i+1,j+1]) 

    return r 

def TheSetYkMMPrime(k,m1,m2): # m1, m2 are multisegments
 
    r=[]
    for i in [0..len(m1)-1]:
        for j in [0..len(m2)-1]:
            if IsSegementAkPrecB(k, LeftShiftOfSegment(m1[i]), m2[j] )==1:
                #r.append([i,j]) 
                r.append([i+1,j+1]) 
    return r 

def LeftShiftOfSegment(A): # 
 
    r=[A[0]-1, A[1]-1]

    return r
 
def IsSegementAkPrecB(k,a,b): # is a \prec_k b?
 
    r=0

    if IsSegementAPrecB(a,b)==1 and b[1]-a[0]<k:
        r=1
     
    return r 

def IsSegementAPrecB(a,b):

    r=0

    if IsAInSegment(a[0], b)==0 and IsAInSegment(LeftShiftOfSegment(b)[0], a)==1 and IsAInSegment(b[1], a)==0:
        r=1 

    return r

def IsAInSegment(a, A): # is a in segment A?
 
    r=0

    if a>=A[0] and a<=A[1]:
        r=1
     

    return r


def CollectNonZeros2(l):  
    r=[]
    for i in l:
        if i[0]!=0 and i[1]!=[]:
            r.append(i)
    return r

def LongestPerm(A,Bsorted,B):
    k = len(A)
    S = set([(A[i],B[i]) for i in range(k)])
    W = WeylGroup('A'+str(k-1), prefix = 's')
    #print "warning: doing hard-coded"
    #s = W.simple_reflections()
    #return s[1]*s[3]*s[4]*s[5]*s[3]*s[4]*s[3]*s[7]
    winner = W.one()
    for w in W:
        wstr = w.inverse().to_permutation_string()
        if set([(A[int(wstr[i])-1],Bsorted[i]) for i in range(k)]) == S:
            #print wstr
            if w.length()>winner.length():
                winner = w
    return winner

def TableauToFundamentalTableauxSLkOneColumn(l1): # l1 is a tableau with one column, l1 is in list form
 
    r=[] 
    l=sorted(l1)
    for i in [0..len(l)-1]:
        if l[i]!=i+1 and l[i]<10000:
            t1=ListAToN(1,i)
            t1=append(t1,l[i])
            r=append(r,t1)
      

    return r

def ReverseList(l):
    r=[]
    for i in range(len(l)-1,-1,-1):
        r=r+[l[i]]
    return r


def TableauToFundamentalTableauxSLk(Y,l): # l is a tableau T, l[i] is column of i of T
 

    r=[]

    for i in [0..len(l)-1]: 
        r=UnionOfLists(r, TableauToFundamentalTableauxSLkOneColumn(l[i]))
    r=sorted(r)
    #print(r)
    r=ReverseList(r)

    return r

def TableauToListsIJSLk(Y,a): # I is the set of the lengths of columns, J is the set of largest elements in each column
    a1=TableauToFundamentalTableauxSLk(Y,a);
    r1=[]
    for i in a1:
        r1.append(len(i))
    r2=[]
    for i in a1:
        r2.append(max(i)) 
    r=[r1,r2]
    return r

def FundamentalTableau(l,a):
    r=[]
    for i in [1..l-1]:
        r.append(i)
    r.append(a)
    
    return r


def TableauToListsIJSortedSLk(Y,a): # I is the set of the first element, J is the list of missing elements
    a1=TableauToFundamentalTableauxSLk(Y,a);
    r1=[]
    for i in a1:
        r1.append(len(i))
    r2=[]
    for i in a1:
        r2.append(max(i)) 
    r=[sorted(r1),sorted(r2)]
    return r
    
def NumberOfA(l,a):
    r=0;
    for i in l:
        if i==a:
            r=r+1
    return r

def IsSemistandardTableauInRows(a): # a[i]'s are rows
    r=1
    
    for i in [0..len(a)-2]:
        if len(a[i])<len(a[i+1]):
            r=0
            return r
    for i in [0..len(a)-1]:
        for j in [0..len(a[i])-2]:
            if a[i][j+1]<a[i][j]:
                r=0
                return r
    r1=TransposeTableauSLk(a)
    for i in [0..len(r1)-1]:
        for j in [0..len(r1[i])-2]:
            if r1[i][j+1]<=r1[i][j]:
                r=0
                return r
        
    return r
    
def ListToPolynomialInQ(l, q):
    r=0

    for i in [0..len(l)-1]:
        r=r+l[i]*q^(i-1)
    return r
    
def ListToPolynomialInQList(l, q):
    r=[]
    for i in l:
        r.append(ListToPolynomialInQ(i, q))
    return r
    
def IsSubsetAsMultiset(a,l):
    r=1
    for i in a:
        t1=NumberOfA(a,i)-NumberOfA(l,i)
        #print(t1)
        if t1>0:
            r=0
            break
    return r

def SymmetricGroupActionOnTableauxComputeChSLk(Y,k,w1,T): # SLk, w is an element in Sm written in one-line notation, nmlen(ii)
    r1=TableauToListsIJSortedSLk(Y,T) # sorted ii, jj
    ii=r1[0]
    jj=r1[1]
    w=ShiftByA(w1,-1)
    #print(ii)
    #print(jj)
    
    m=len(ii)
    r=[]
        
    #print(m)   
    #print('w',w)
    for b in range(m):
        #print('b',b)
        if jj[b]>=ii[w[b]] and jj[b]<=ii[w[b]]+k:
            #print('ii,jj,w, fundamental tableau:', ii,jj,w, FundamentalTableau(ii[w[b]], jj[b]))
            r.append( FundamentalTableau(ii[w[b]], jj[b]) )
        else:
            r=[]
            break;
    r=sorted(r)
    return r

def CollectNonZeros(l):
    r=[]
    for i in l:
        if i[0]!=0:
            r.append(i)
    return r


def IsInList(i,l):
    if (i in l)==True:
        r=1
    else:
        r=0
    return r

def combineLikeTerms3(l):

    r=[]
    l2=l
    
    #print('l',l)
    temp2=[]

    for i in l2:

        if (IsInList(removeAnElementInList(0,i), temp2)==0): 
            t=0
            for j in l2:
                if removeAnElementInList(0,j)==removeAnElementInList(0,i):
                    t=t+j[0]
              
            s=i
            s[0]=simplify(t)
            r.append(s)
            temp2.append(removeAnElementInList(0,i))
    
    r1=r
    r=[]
    for i in r1:
        if i[0]!=0:
            r.append(i)

    return r


def ShiftByA(l,a):
    r=[]
    for i in l:
        r.append(i+a)
    return r

def ModuleToTableauSLk(Y,l,k): # SLk
    t1=ModuleToMultiSegment(Y,l)
    r=MultisegmentToTableauSLk(k,t1)
    return r

def MultisegmentToTableauSLkOneSegment(k, m): # SLk
 
    a=m[0]
    b=m[1]
    t1=ListAToN(1, -a)
    t1.append(k-b)
    r=[t1]

    return r


def MultisegmentToTableauSLk(k, l): # SLk

    r=[]
    if l!=[]:
        for i in l:
            t1=MultisegmentToTableauSLkOneSegment(k, i)
            #print('t1', t1)
            r=UnionOfTwoTableauxSLk(r, t1)
            
        #print('r', r)
             
        r=RemoveTrivialTableauSLk(r,k) 
        #print(r)
    return r

def PluckersWhichAreOneSLk(k): # SLk 1, 12, 123, ..., are trivial
    r=[]
    for i in [1..k]:
        t1=[]
        for j in [1..i]:
            t1.append(j)
        r.append(t1)
    return r

def IsTableauAFactorOfBSLk(a,b):
    
    r1=a
    r2=b

    r=1;
    for i in range(len(r1)):
        if IsSubsetAsMultiset(r1[i], r2[i])==0:
            r=0
            break
    return r

def RemoveTrivialTableauSLk(a1,k): # SLk
    r=a1
    t2=PluckersWhichAreOneSLk(k);
    sn=1
    while sn!=0:
        r1=r;
        for i in range(len(t2)):
            #print(r, t2[i])
            if r==[]:
                break
            elif IsTableauAFactorOfBSLk([t2[i]], r)==1:
                p1=TableauDivisionSLkResultInRows(r, [t2[i]])
                #print('p1',p1)
                if IsSemistandardTableauInRows(p1)==1:
                    r=TableauDivisionSLk(r, [t2[i]])
                
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1
        
    
    return r

def TableauToFundamentalTableauxSLkOneColumn(l1): # l1 is a tableau with one column, l1 is in list form
 
    r=[] 
    l=sorted(l1)
    for i in [0..len(l)-1]:
        if l[i]!=i+1 and l[i]<10000:
            t1=ListAToN(1,i)
            t1=append(t1,l[i])
            r=append(r,t1)
      

    return(r)


def TableauToFundamentalTableauxSLk(Y,l): # l is a tableau T, l[i] is column of i of T
 

    r=[]

    for i in [0..len(l)-1]: 
        r=UnionOfLists(r, TableauToFundamentalTableauxSLkOneColumn(l[i]))
    r=sorted(r)
    #print(r)
    r=ReverseList(r)

    return(r) 

def TableauToModuleSLk(Y,k,l): # l is a tableau T, l[i] is column of i of T, SLk
    
    t1=TableauToFundamentalTableauxSLk(Y,l)
    #print(t1)
 
    r=[]
    for i in t1:
        r.append(FundamentalTableauToYiaSLk(i,k)) 

    return r

def FundamentalTableauToYiaSLk(a1,k): # SLk, T_{l,a}, length l, last entry a, T_{l,a}=T^{l+k-a,l} \sim Y_{l+k-a,k-a-l}, weight of T_{l,a} is omiga_(l+k-a)
 
    r=[]
    rank=k-1
    n=rank
    for i in [1..n]:
        r=append(r,0)
        
    a=sorted(a1)
    l=len(a)
    t1=a[len(a)-1]
    r=[l+k-t1,k-t1-l]
    
    return r

def append(l,i):
    r=l
    r.append(i)
    return r


def UnionOfLists(a,b):
    r=a+b
    return r

def UnionOfListsMany(l):
    r=[]
    for i in l:
        r=r+i
    return r

def TransposeTableauSLk(a): # a[i] are columns
    r=[]
    for j in [0..len(a[0])-1]:
        t1=[]
        for i in [0..len(a)-1]:
            if j<len(a[i]):
                t1.append(a[i][j])
        r.append(t1)

    return r

def ListAToN(a,n):
    r=list(range(a,n+1))
    return r

def IsPartition1GreaterThanOrEqualPartition2(p1,p2): # p1,p2 are partitions
    l1=p1
    l2=p2

    n=max(len(p1), len(p2))
    for i in [len(p1)..n]:
        l1.append(0) 
    for i in [len(p2)..n]: 
        l2.append(0) 

    r=1 # p1 >= p2

    for i in [1..n]: 
        t1=0
        t2=0
        for j in [0..i-1]:
            t1=t1+l1[j]
            t2=t2+l2[j] 
        if t1<t2: 
            r=0 # p1 is not greater or equal to p2
            break 
    return r

def comparePartitions(p1,p2): # p1,p2 are partitions
    t1=IsPartition1GreaterThanOrEqualPartition2(p1,p2)
    t2=IsPartition1GreaterThanOrEqualPartition2(p2,p1)

    if t1==1: 
        r=1  # p1>=p2
    elif t2==1: 
        r=-1 # p1<=p2
    else:
        r=0 # not comparable 
    return r

def IsTableauxP1GreaterEqualP2SLkU(P1,P2): # P1,P2 are tableaux

    n=-1000
    for i in UnionOfListsMany(P1):
        if n<i:
            n=i
    for i in UnionOfListsMany(P2):
        if n<i:
            n=i 

    r=1 # P1>=P2

    for a in [1..n]: 
        t1=TableauRestrictToElementsLessOrEqualAToPartition(P1,a)
        t2=TableauRestrictToElementsLessOrEqualAToPartition(P2,a)
        r1=comparePartitions(t1,t2)
        if r1==0 or r1==-1: 
            r=0 # P1 is not greater or equal to P2
            break 

    return r

def TableauToMatrixTakeRowsSLkMoreGeneral(a): # send a tableau to a matrix, a[i] are rows
    if a==[]:
        r=[]
    else:
        m=len(a)  # number of rows
        n=len(a[0])
        for i in [1..len(a)-1]:
            if n<len(a[i]):
                n=len(a[i])
        
        r=Matrix(m,n)
        for i in range(m):
            for j in range(n):
                if j<len(a[i]):
                    r[i,j]=a[i][j]
                else:
                    r[i,j]=-10000
    return r

def TableauToMatrixTakeColumnsSLkMoreGeneral(a): # send a tableau to a matrix, a[i] are columns
    
    if a==[]:
        r=[]
    else:
        n=len(a)  # number of columns
        m=len(a[0])
        for i in [1..len(a)-1]:
            if m<len(a[i]):
                m=len(a[i])
        
        r=Matrix(m,n)
        for i in range(m):
            for j in range(n):
                if i<len(a[j]):
                    r[i,j]=a[j][i]
                else:
                    r[i,j]=-10000
    return r 

def TableauToMatrixTakeRowsSLkListMoreGeneral(a): # SLk, a[i] are columns of tableau
    r=[]
    for i in a:
        r.append(TableauToMatrixTakeRowsSLkMoreGeneral(i))
    return r

def TableauToMatrixTakeColumnsSLkListMoreGeneral(a): # SLk, a[i] are columns of tableau
    r=[]
    for i in a:
        r.append(TableauToMatrixTakeColumnsSLkMoreGeneral(i))
    return r

def TableauToPartition(P):

    r=[]
    t1=TableauToMatrixTakeColumnsSLkMoreGeneral(P)
    for i in [0..t1.nrows()-1]: 
        t2=0
        for j in [0..t1.ncols()-1]: 
            if t1[i,j]!=-10000:
                t2=t2+1 
        r.append(t2) 
    return r

def TableauRestrictToElementsLessOrEqualAToPartition(P,a):

    t1=TableauRestrictToElementsLessOrEqualA(P,a)
    r=TableauToPartition(t1)

    return r

def compareWeightsTableauxSLkU(P1,P2): # P1,P2 are tableaux

    t1=IsTableauxP1GreaterEqualP2SLkU(P1,P2)
    t2=IsTableauxP1GreaterEqualP2SLkU(P2,P1)

    if t1==1:  
        r=1 # P1>=P2
    elif t2==1: 
        r=-1 # P1<=P2
    else:
        r=0 # P1, P2 not comparable
     
    return r

def TableauRestrictToElementsLessOrEqualA(P,a):

    r=[]
    for i in P: 
        t1=[]
        for j in i: 
            if j<=a:
                t1.append(j) 
        r.append(t1) 
    return r

# mutation for base affine space, C[SL_k]^U, and C[N]

def InitialCluster_SLk(kk): # initial cluster for SL_kk, only works for kk <= 10

    # quivers are from Keller's mutation app

    if kk==2: 
        mat=Matrix([[0,-1,1],[1,0,-1],[-1,1,0]]) # SL2
    elif kk==3: 
        mat=Matrix([[0,-1,1,0,0,0],[1,0,-1,-1,1,0],[-1,1,0,0,-1,1],[0,1,0,0,-1,0],[0,-1,1,1,0,-1],[0,0,-1,0,1,0]]) # SL3
    elif kk==4:
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0],[0,-1,1,1,0,-1,0,-1,1,0],[0,0,-1,0,1,0,0,0,-1,1],[0,0,0,1,0,0,0,-1,0,0],[0,0,0,-1,1,0,1,0,-1,0],[0,0,0,0,-1,1,0,1,0,-1],[0,0,0,0,0,-1,0,0,1,0]]) # SL4
    elif kk==5: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0]]) # SL5
    elif kk==6: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0]]) # SL6
    elif kk==7: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0]]) # SL7
    elif kk==8: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0]]) # SL8
    elif kk==9: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0]]) # SL9
    elif kk==10: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0]]) # SL10

    vertices0=[]
    n=kk

    for i in [1..n]: 
        for j in [1..i]: 
            t3=ListAToN(j, j+n-i)
            vertices0.append(t3) 
            
    verticesTableaux=[] # Tableaux are represented by matrices
    for i in [0..len(vertices0)-1]:
        verticesTableaux.append([0, [vertices0[i]], i]) # [vertices0[i]] is an one column tableau
        
    mat1 = mat
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r
    
def UnionOfTwoTableauxSLk(a,b): # a[i], b[i] are columns
    if a==[]:
        r=b
    elif b==[]:
        r=a
    else:
        r1=TransposeTableauSLk(a)
        r2=TransposeTableauSLk(b)
        
        m=max(len(r1), len(r2))
        #print('m,len(r1),len(r2)',m,len(r1),len(r2))
        r3=[]
        for i in [0..m-1]:
            #print(i,len(r1),len(r2))
            if i<len(r1) and i<len(r2):
                t2=r1[i]+r2[i]
                #print('t2',t2)
                t2=sorted(t2)
            elif i>=len(r1) and i<len(r2):
                    t2=sorted(r2[i])
            elif i<len(r1) and i>=len(r2):
                    t2=sorted(r1[i])
            r3.append(t2)
        #print('r3',r3)
        r=TransposeTableauSLk(r3)
    return r

def PowerOfTableaux_SLk(T, n): # T=[[1,2,3],[2,5]]
    if n==1:
        r=T
    elif n>1:
        r=T
        for i in range(n-1):
            r=UnionOfTwoTableauxSLk(r, T)
    return r
    
def computeEquationsForModulesTableaux_SLk(variable2, mat, k, kk): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableauxSLk(newVariable, PowerOfTableaux_SLk(variable[i][1], mat[i,k]) )
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableauxSLk( newVariable2, PowerOfTableaux_SLk(variable[i][1], -mat[i,k]) )
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableauxSLkU(newVariable, newVariable2)
    
    if t1==1: 
        variable[k][1]=TableauDivisionSLk(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivisionSLk(newVariable2, variable[k][1])  
    
    #print(variable[k][1])
    clusterVariables=TableauToMatrixTakeColumnsSLkMoreGeneral(variable[k][1])
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def MatrixToTableauTakeColumnsSLk(a): 
 

    n=a.nrows()
    m=a.ncols()

    r=[]
    for i in [0..m-1]:
        t2=[]
        for j in [0..n-1]:
            if a[j,i]>=0:
                t2=append(t2,a[j,i])
        
        r=append(r, t2)
    
    #r=SemistandardTableau(r)
   
    return r

def ll_perms_SLk(lli, kk, max_column, repeat): # SL_kk
    b1=[]
    IC=InitialCluster_SLk(kk)
    mat1=IC[0]
    vertices1=IC[1]

    ll=list(map(lambda x: x - 1, lli))
    
    #print(IC)
    
    mutationSequence=[]
    for j1 in [1..repeat]: ###why do this?, just to repeat the same sequence of mutations, it will give more cluster variables, without this loop is also ok
        mutationSequence=mutationSequence+ll 
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesTableaux_SLk(vertices1, mat1, mutationSequence[j], kk)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 

        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux_SLk(vertices1, mat1, mutationSequence[j], kk)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            #print(vertices1[1])
            b1.append(MatrixToTableauTakeColumnsSLk(vertices1[1]))

    return b1

def TableauDivisionSLk(a,b): # a[i], b[i] are columns
    r1=TableauDivisionSLkResultInRows(a,b)
        
    r=TransposeTableauSLk(r1)
        
    return r

def TableauDivisionSLkResultInRows(a,b): # a[i], b[i] are columns
    r1=TransposeTableauSLk(a)
    r2=TransposeTableauSLk(b)
    
    r3=[]
    m=max(len(r1),len(r2))
    for i in range(m):
        if i<len(r1) and i<len(r2):
            r3.append(sorted(SetDifferenceListDifference(r1[i],r2[i])))
        elif i<len(r1) and i>=len(r2):
            r3.append(r1[i])
        elif i>=len(r1) and i<len(r2):
            r3=[]
            print('tableau not divisible')
            break
        
    r=r3
        
    return r

def TableauToMatrixTakeRows(a): 
    # a[i] are rows of the tableau a
    # transform a tableau to matrix form
    m=len(a)
    n=len(a[0])
    r=Matrix(m,n)
    for i in range(m):
        for j in range(n):
            r[i,j]=a[i][j]
    return r

def PromotionOfTableauNTimes(N,T1,n):
    # promotion of a tableau N times
    r=[T1]
    T=T1
    for i in range(N-1):
        T=T.promotion(n-1)
        r.append(T)
    return r 

def PromotionOfTableauNTimesInMatrix(N,T1,n):
    # promotion of tableau n times, T1 is in matrix form
    t1=MatrixTakeRows(T1)
    t2=SemistandardTableau(t1)
    r1=PromotionOfTableauNTimes(N,t2,n)
    r=[]
    for i in r1:
        r.append(TableauToMatrixTakeRows(i))
        
    return r 

def PluckerToMinimalAff(a1):
    # translate a Plucker coordinate to highest weight monomial
    r=[]
    a=sorted(a1)
    n=len(a)
    for i in range(n-1,0,-1):
        r.append(a[i]-a[i-1]-1)
    r.append(a[0]-1)

    return r

def InitialCluster(rank,n):  
    # initial cluster from Gr(rank, n)
    sizeColumn=n-rank
    k=sizeColumn    
    k1=rank
    p1=k*rank+1
    mat=Matrix(p1,p1)
    for i in range(p1): 
        i1=i+1
        if i1==1: 
            mat[i,i+k+1]=1
            mat[i, p1-1]=1
            mat[i,i+k]=-1
            mat[i, i+1]=-1
        elif i1>=2 and i1<=k-1: 
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1==k: 
            mat[i,i-1]=1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+k+1]=1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k >=2 and i1 % k<=k-1:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1>=2*k and i1<=(rank-1)*k and i1 % k==0:
            mat[i,i-k-1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1>(rank-1)*k and i1<p1 and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+1]=-1
        elif i1>=(rank-1)*k+2 and i1<rank*k:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==rank*k:
            mat[i,i-k-1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==p1:
            mat[i,0]=-1

    vertices0=[]

    for j in range(k1-1,-1,-1):
        for i in range(k1,k1+k):
            t1=list(range(1,j+1))
            t2=list(range(i-k1+j+2,i+2))
            t3=t1+t2
            vertices0.append(t3)

    vertices0.append(list(range(1,k1+1)))  
    verticesTableaux = [] # Tableaux are represented by matrices
    for i in range(len(vertices0)):
        verticesTableaux.append([0, [vertices0[i]], i]) # [vertices0[i]] is an one column tableau
    mat1 = Matrix(p1,p1)
    for i in range(p1):
        for j in range(p1):
            mat1[i,j]=mat[i,j]
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r


def TableauExpansionsInMatrixHalf(l,b,c): 
    # l is tableau in matrix form, b is the content of l, c is a list of numbers
    # replace a_1<...<a_m in l by c_1<...<c_m
    r=[]
    m=l.nrows()
    n=l.ncols()
    r=Matrix(m,n)
    for i in range(m):
        for j in range(n):
            t1=b.index(l[i,j])+1
            r[i,j]=c[t1-1]
    return r

def TableauExpansionsInMatrix(l,n): 
    # l is tableau in matrix form
    # replace a_1<...<a_m in l by c_1<...<c_m
    r1=ContentOfTableau(l)
    m=len(r1)
    r2=list(itertools.combinations(list(range(1,n+1)), m)) 
    r=[]
    for i in r2:
        t1=TableauExpansionsInMatrixHalf(l,r1,i)
        r.append(t1)
    return r

def TableauExpansionsInMatrixList(l,n): 
    # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        r=r+TableauExpansionsInMatrix(i,n)
    r=removeDuplicates2(r)  
    
    return r

def ContentOfTableau(l): 
    # l is tableau
    # compute the content of l, with multiplicities
    r=[]
    for i in l:
        for j in i:
            r.append(j)
    #r=np.unique(r,axis=0)
    r=removeDuplicatesListOfLists(r)
    r=sorted(r)
    return r

def immutabilize(m):
    M = copy(m)
    M.set_immutable()
    return M

def ChangeListOfMatricesToSetOfMatrices(S):
    r={immutabilize(i) for i in S}
    return r

def removeAnElementInList(i, l):
    r=[]
    for j in range(len(l)):
        if (j!=i):
            r.append(l[j])
    
    return r

def removeDuplicates(l):
    # remove duplicates
    # it is slow whn l is large
    r=[]
    for i in l:
        if (i in r)==False:
            r.append(i)
    return r

def removeDuplicates2(l): 
    # remove duliplictes
    # vary fast, for matrices
    t1=ChangeListOfMatricesToSetOfMatrices(l)
    r=list(dict.fromkeys(t1))
    return r

def removeDuplicatesListOfLists(l): 
    # very fast
    l.sort()
    r=list(l for l,_ in itertools.groupby(l))

    return r

def SetDifference2(a,b):
    # take different of two lists a, b
    t1=ChangeListOfMatricesToSetOfMatrices(a)
    t2=ChangeListOfMatricesToSetOfMatrices(b)
    r=t1.difference(t2)
    return r

def SetDifferenceListDifference(A,B): 
    # A-B, can have duplicate elements
    # take different of two lists A, B, count multiplicites
    r=[]
    r1=list(set(A))
    for i in r1:
        t1=A.count(i)-B.count(i)
        #print(t1)
        for j in range(1,t1+1):
            r.append(i)
            
    return r

def TableauToMatrix(a):
    # transform a tableau to matrix form
    m=len(a)
    n=len(a[0])
    r=Matrix(n,m)
    for i in range(n):
        for j in range(m):
            r[i,j]=a[j][i]
    return r

def MatrixTakeRows(a):
    # take rows of a matrix to get a list
    n=a.nrows()
    m=a.ncols()
    r=[]
    for i in range(n):
        t1=a[[i],list(range(m))]
        t2=[]
        for j in range(m):
            t2.append(t1[0,j])
        r.append(t2)
    return r

def MatrixTakeRowsList(a):
    # function MatrixTakeRows for a list of matrices
    r=[]
    for i in a:
        r.append(MatrixTakeRows(i))
    return r

def TableauDivision(a,b):
    # division of two tableaux a, b, that is removing b from a
    t1=TableauToMatrix(a)
    t2=TableauToMatrix(b)  
    r1=MatrixTakeRows(t1)
    r2=MatrixTakeRows(t2)
    r3=[]
    for i in range(len(r1)):
        r3.append(sorted(SetDifferenceListDifference(r1[i],r2[i])))
    r=[]
    for i in range(len(r3[0])):
        t1=[]
        for j in range(len(r3)):
            t1.append(r3[j][i])
        r.append(t1)
        
    return r

def UnionOfTwoTableaux(a,b):
    t1=a+b
    t2=TableauToMatrix(t1)
    r=[]
    for i in range(t2.nrows()):
        r1=[]
        for j in range(t2.ncols()):
            r1.append(t2[i,j])
        r.append(sorted(r1))
        
    r2=TableauToMatrix(r);
    r=[]
    for i in range(r2.nrows()):
        r1=[]
        for j in range(r2.ncols()):
            r1.append(r2[i,j])
        r.append(sorted(r1))
    return r
        
def PowerOfTableaux(a,n):
    r=[]
    if a!=[] and a!=[[]]:
        for i in range(1,n+1):
            r=UnionOfTwoTableaux(r,a) 
    else:
        r=a 
    return r

def CartanMatrixSelfDefined(typ, rank):
    if typ=='E' and rank==6:
        r=Matrix([[2,0,-1,0,0,0],[0,2,0,-1,0,0],[-1,0,2,-1,0,0],[0,-1,-1,2,-1,0],[0,0,0,-1,2,-1],[0,0,0,0,-1,2]]) # this is the Cartan Matrix in Sage of type E6
    else:
    
        r = Matrix(rank, rank)
        n = rank
        for i in range(n):
            if i + 1 <= n-1:
                r[i, i + 1] = -1
            if 0 <= i - 1:
                r[i, i - 1] = -1
            r[i, i] = 2

        if typ == 'B' or typ == 2:
            r[n-1, n - 2] = -2
        elif typ == 'C' or typ == 3:
            r[n - 2, n-1] = -2
        elif typ == 'D' or typ == 4:
            if n == 2:
                r[0, 1] = 0
                r[1, 0] = 0
            elif 3 <= n:
                r[n - 3, n - 2] = -1
                r[n - 3, n-1] = -1
                r[n - 2, n - 3] = -1
                r[n-1, n - 3] = -1
                r[n - 2, n-1] = 0
                r[n-1, n - 2] = 0
        elif typ == 'E' or typ == 5:
            for k in [[2, 4], [4, 2]]:
                r[k[0], k[1]] = -1
            for k in [[3, 4], [4, 3]]:
                r[k[0], k[1]] = 0
        elif typ == 'F' or typ == 6:
            r[1, 2] = -2
        elif typ == 'G' or typ == 7:
            r[0, 1] = -3
     
    return r 

def compareWeightsTableaux(P1,P2,typ,rank): 
    # a,b are tableaux
    t1=WeightOfTableau(P1)
    t2=WeightOfTableau(P2)
    r=compareWeights2(t1,t2,typ, rank)

    return r

def WeightOfTableau(a): 
    # a[i] are columns of the tableau a
    m=len(a)
    n=len(a[0])
    r=[]
    for i in range(1,n+1):
        r.append(0)
    for i in range(m):
        t1=PluckerToMinimalAff(a[i])
        #r=list(np.array(r)+np.array(t1))
        r=list(map(add, r, t1))
        
    return r

def compareWeights(a, b, typ, rank): 
    # compare two weights
    r=1                             # r=1 means a>=b          
    l=a-b
    c=CartanMatrixSelfDefined(typ, rank)
    for i in range(rank): 
        p=0
        for j in range(rank):
            t1=(transpose(c)^(-1))[j,i]
            p=p+l[j,0]*t1
        if p<0: 
            r=-1              # r=-1 means a is not >= b, it is possible that a<b or a,b are not comparable
            break 
            
    if r==-1:
        for i in range(rank):
            p=0
            for j in range(rank):
                t1=(transpose(c)^(-1))[j,i]
                p=p+l[j,0]*t1
            if p>0: 
                r=0
                break
    return r

def compareWeights2(a,b,typ,rank): 
    # a,b are lists
    n=len(a)
    t1=Matrix(n,1)
    for i in range(n): 
        t1[i,0]=a[i] 
    t2=Matrix(n,1)
    for i in range(n): 
        t2[i,0]=b[i] 
    r=compareWeights(t1,t2,typ,rank)

    return r

def matrixMutation(mat,  k):  
    # matrix mutates at k
    size=mat.nrows()
    r=Matrix(size,size)
    for i in range(size):
        for j in range(size):
            r[i,j]=mat[i,j]
    
    for i in range(size): 
        for j in range(size): 
            
            if k==i or k==j:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j]+1/2*(abs(mat[i,k])*mat[k,j]+mat[i,k]*abs(mat[k,j]))
     
    return r

def ExtendSetOfTableauxToContainPromotions(l,n): 
    # l is a list of tableaux 
    # extend the set l to include their promotions
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimes(n,i,n)
        r=r+t1 
    r=np.unique(r,axis=0)

    return r

def ExtendSetOfTableauxToContainPromotionsInMatrix(l,n): 
    # l is a list of tableaux in matrix form
    # extend the set l to include their promotions
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicates2(r)
    
    return r

def TableauxToListOfTimesOfOccurrenceOfNumbers(a):
    # compute occurrences of numbers in tableau a
    r=[]
    n=a.nrows() 
    m=a.ncols() 
    r1=[]
    for i in range(n):  
        for j in range(m): 
            r1.append(a[i,j]) 
    for k in range(1,max(r1)+1):
        t1=0
        for i in r1:
            if i==k:
                t1=t1+1 
        r.append(t1)  
    return r

def TableauxToListOfTimesOfOccurrenceOfNumbersLengthN(a,N):
    r=[]
    n=a.nrows() 
    m=a.ncols() 
    r1=[]
    for i in range(n):  
        for j in range(m): 
            r1.append(a[i,j]) 
    for k in range(1,N):
        t1=0
        for i in r1:
            if i==k:
                t1=t1+1 
        r.append(t1)
    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersLengthNWithContentLessOrEquN(a,N): 
    # compute the occurrences of numbers in i for those i in a such that the numbers in i is less or equal to N
    r=[]
    n=a.nrows() 
    m=a.ncols() 
    r1=[]
    for i in range(n):  
        for j in range(m): 
            r1.append(a[i,j])       
    if max(r1)<=N:
        for k in range(1,N):
            t1=0
            for i in r1:
                if i==k:
                    t1=t1+1 
            r.append(t1)
    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsList(a):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbers(t1)
 
    return r

def TableauxToListOfTimesOfOccurrenceOfNumbersLengthNTableauIsList(a,N):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbersLengthN(t1,N)
 
    return r


def computeEquationsForModulesTableaux(variable2, mat, k, typ, rank): 
    # mutation of Grassmannian cluster variables
    # variable2=(variables on quiver, cluster variables obtained so far)
    variable1=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    variable=variable1

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableaux( newVariable, PowerOfTableaux(variable[i][1], mat[i,k]) )
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableaux( newVariable2, PowerOfTableaux(variable[i][1], -mat[i,k]) )
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableaux(newVariable, newVariable2,typ,rank)
 
    if t1==1: 
        variable[k][1]=TableauDivision(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivision(newVariable2, variable[k][1])  
        
    clusterVariables=TableauToMatrix(variable[k][1]) 
    
    r=[variable, clusterVariables]

    return r


def ll_perms(lli,typ,rank,max_column,n,repeat): 
    #Function for multiprocessing
    b1=[]
    IC=InitialCluster(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    ll=list(map(lambda x: x - 1, lli))
    
    mutationSequence=[]
    for j1 in [1..repeat]: # repeat the same sequence of mutations, it will give more cluster variables
        mutationSequence=mutationSequence+ll 
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 

        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) # if encounter a cluster variable with too large number of columns, we mutate again to remove it
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            b1.append(vertices1[1]) 
            
    b1=removeDuplicates2(b1)
    b1=ExtendSetOfTableauxToContainPromotionsInMatrix(b1,n)
    b1=TableauExpansionsInMatrixList(b1, n)
    
    return b1

In [2]:
####################################################################
# compute only cluster variables, SLk
if __name__ == '__main__':
    kk = 6   #...for SL_kk
    max_column = 4 #...obtain only tableaux with number of columns less or equal to max_column
    max_step = 23     #...this number controls the length of random mutation sequence, in order to obtain all cluster variables with number of columns less or equal to a fixed number, we need to put the number max_step sufficiently large
    checkpoint = 3   #...if after check_point steps, the number elements in b2 is not increasing, then stop
    repeat=25
    
    b1=[]
    
    #Run generation 
    
    if kk==3:   # SL3
        ll0=[5]
    elif kk==4:
        ll0=[5,8,9]
    elif kk==5:
        ll0=[5,8,9,12,13,14]
    elif kk==6:
        ll0=[9, 18, 19, 13, 5, 17, 8, 20, 12, 14]
    elif kk==7:
        ll0=[5,8,9,12,13,14,17,18,19,20,23,24,25,26,27]
    elif kk==8:
        ll0=[5,8,9,12,13,14,17,18,19,20,23,24,25,26,27,30,31,32,33,34,35]
    elif kk==9:
        ll0=[5,8,9,12,13,14,17,18,19,20,23,24,25,26,27,30,31,32,33,34,35,38,39,40,41,42,43,44]
    elif kk==10:
        ll0=[5,8,9,12,13,14,17,18,19,20,23,24,25,26,27,30,31,32,33,34,35,38,39,40,41,42,43,44,47,48,49,50,51,52,53,54]

            
    sn=0
    num=0
    
    
    while (1):  #...have split the ComputeClusterVariablesInGrkn function up into part we wish to parallelise 'll_perms' and the remainder
        sn=sn+1

        lls = [np.random.permutation(ll0) for iii in range(max_step)]
        for i in lls:
            t1=ll_perms_SLk(i, kk, max_column, repeat)
            b1=b1+t1
        
        b1=removeDuplicatesListOfLists(b1)
        
        if sn%checkpoint==1:
            print('sn, len(b1)', sn, len(b1))
            if num==len(b1) and num!=0:
                break
            else:
                num=len(b1)



sn, len(b1) 1 504
sn, len(b1) 4 553
sn, len(b1) 7 554
sn, len(b1) 10 554


In [3]:
# this is the result of cluster variables in C[SL_5/N] that I obtained by the above computations 
ClusterVariablesSL5=[[[1, 2, 3, 5]], [[1, 2, 3, 5], [1, 4]], [[1, 2, 3, 5], [2, 4]], [[1, 2, 3, 5], [3, 4]], [[1, 2, 3, 5], [4]], [[1, 2, 4]], [[1, 2, 4], [3]], [[1, 2, 4], [3, 5]], [[1, 2, 4, 5]], [[1, 2, 4, 5], [3]], [[1, 2, 4, 5], [3, 4]], [[1, 2, 4, 5], [3, 5]], [[1, 2, 5]], [[1, 2, 5], [3]], [[1, 2, 5], [4]], [[1, 3]], [[1, 3, 4]], [[1, 3, 4, 5]], [[1, 3, 5]], [[1, 3, 5], [2, 4]], [[1, 3, 5], [4]], [[1, 4]], [[1, 4, 5]], [[1, 5]], [[2]], [[2, 3]], [[2, 3, 4]], [[2, 3, 5]], [[2, 3, 5], [4]], [[2, 4]], [[2, 4, 5]], [[2, 5]], [[3]], [[3, 4]], [[3, 5]], [[4]]]

In [5]:
# not include frozen variables
Rank1ClusterVariablesSL6=[[[1, 2, 3, 4, 6]], [[1, 2, 3, 5]], [[1, 2, 3, 5, 6]], [[1, 2, 3, 6]], [[1, 2, 4]], [[1, 2, 4, 5]], [[1, 2, 4, 5, 6]], [[1, 2, 4, 6]], [[1, 2, 5]], [[1, 2, 5, 6]], [[1, 2, 6]], [[1, 3]], [[1, 3, 4]], [[1, 3, 4, 5]], [[1, 3, 4, 5, 6]], [[1, 3, 4, 6]], [[1, 3, 5]], [[1, 3, 5, 6]], [[1, 3, 6]], [[1, 4]], [[1, 4, 5]], [[1, 4, 5, 6]], [[1, 4, 6]], [[1, 5]], [[1, 5, 6]], [[1, 6]], [[2]], [[2, 3]], [[2, 3, 4]], [[2, 3, 4, 5]], [[2, 3, 4, 6]], [[2, 3, 5]], [[2, 3, 5, 6]], [[2, 3, 6]], [[2, 4]], [[2, 4, 5]], [[2, 4, 5, 6]], [[2, 4, 6]], [[2, 5]], [[2, 5, 6]], [[2, 6]], [[3]], [[3, 4]], [[3, 4, 5]], [[3, 4, 6]], [[3, 5]], [[3, 5, 6]], [[3, 6]], [[4]], [[4, 5]], [[4, 6]], [[5]]]
Rank2ClusterVariablesSL6=[[[1, 2, 3, 4, 6], [1, 2, 5]], [[1, 2, 3, 4, 6], [1, 3, 5]], [[1, 2, 3, 4, 6], [1, 4, 5]], [[1, 2, 3, 4, 6], [1, 5]], [[1, 2, 3, 4, 6], [2, 3, 5]], [[1, 2, 3, 4, 6], [2, 4, 5]], [[1, 2, 3, 4, 6], [2, 5]], [[1, 2, 3, 4, 6], [3, 4, 5]], [[1, 2, 3, 4, 6], [3, 5]], [[1, 2, 3, 4, 6], [4, 5]], [[1, 2, 3, 4, 6], [5]], [[1, 2, 3, 5], [1, 4]], [[1, 2, 3, 5], [1, 4, 6]], [[1, 2, 3, 5], [2, 4]], [[1, 2, 3, 5], [2, 4, 6]], [[1, 2, 3, 5], [3, 4]], [[1, 2, 3, 5], [3, 4, 6]], [[1, 2, 3, 5], [4]], [[1, 2, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 4]], [[1, 2, 3, 5, 6], [1, 4, 5]], [[1, 2, 3, 5, 6], [1, 4, 6]], [[1, 2, 3, 5, 6], [2, 4]], [[1, 2, 3, 5, 6], [2, 4, 5]], [[1, 2, 3, 5, 6], [2, 4, 6]], [[1, 2, 3, 5, 6], [3, 4]], [[1, 2, 3, 5, 6], [3, 4, 5]], [[1, 2, 3, 5, 6], [3, 4, 6]], [[1, 2, 3, 5, 6], [4]], [[1, 2, 3, 5, 6], [4, 5]], [[1, 2, 3, 5, 6], [4, 6]], [[1, 2, 3, 6], [1, 4]], [[1, 2, 3, 6], [1, 5]], [[1, 2, 3, 6], [2, 4]], [[1, 2, 3, 6], [2, 5]], [[1, 2, 3, 6], [3, 4]], [[1, 2, 3, 6], [3, 5]], [[1, 2, 3, 6], [4]], [[1, 2, 3, 6], [4, 5]], [[1, 2, 3, 6], [5]], [[1, 2, 4], [3]], [[1, 2, 4], [3, 5]], [[1, 2, 4], [3, 5, 6]], [[1, 2, 4], [3, 6]], [[1, 2, 4, 5], [3]], [[1, 2, 4, 5], [3, 4]], [[1, 2, 4, 5], [3, 4, 6]], [[1, 2, 4, 5], [3, 5]], [[1, 2, 4, 5], [3, 5, 6]], [[1, 2, 4, 5], [3, 6]], [[1, 2, 4, 5, 6], [3]], [[1, 2, 4, 5, 6], [3, 4]], [[1, 2, 4, 5, 6], [3, 4, 5]], [[1, 2, 4, 5, 6], [3, 4, 6]], [[1, 2, 4, 5, 6], [3, 5]], [[1, 2, 4, 5, 6], [3, 5, 6]], [[1, 2, 4, 5, 6], [3, 6]], [[1, 2, 4, 6], [1, 3, 5]], [[1, 2, 4, 6], [1, 5]], [[1, 2, 4, 6], [2, 3, 5]], [[1, 2, 4, 6], [2, 5]], [[1, 2, 4, 6], [3]], [[1, 2, 4, 6], [3, 4]], [[1, 2, 4, 6], [3, 5, 6]], [[1, 2, 4, 6], [3, 6]], [[1, 2, 4, 6], [4, 5]], [[1, 2, 4, 6], [5]], [[1, 2, 5], [3]], [[1, 2, 5], [3, 6]], [[1, 2, 5], [4]], [[1, 2, 5], [4, 6]], [[1, 2, 5, 6], [3]], [[1, 2, 5, 6], [3, 5]], [[1, 2, 5, 6], [3, 6]], [[1, 2, 5, 6], [4]], [[1, 2, 5, 6], [4, 5]], [[1, 2, 5, 6], [4, 6]], [[1, 2, 6], [3]], [[1, 2, 6], [4]], [[1, 2, 6], [5]], [[1, 3, 4, 6], [1, 5]], [[1, 3, 4, 6], [2, 3, 5]], [[1, 3, 4, 6], [2, 4, 5]], [[1, 3, 4, 6], [2, 5]], [[1, 3, 4, 6], [3, 5]], [[1, 3, 4, 6], [4, 5]], [[1, 3, 4, 6], [5]], [[1, 3, 5], [2, 4]], [[1, 3, 5], [2, 4, 6]], [[1, 3, 5], [4]], [[1, 3, 5], [4, 6]], [[1, 3, 5, 6], [2, 4]], [[1, 3, 5, 6], [2, 4, 5]], [[1, 3, 5, 6], [2, 4, 6]], [[1, 3, 5, 6], [4]], [[1, 3, 5, 6], [4, 5]], [[1, 3, 5, 6], [4, 6]], [[1, 3, 6], [2, 4]], [[1, 3, 6], [2, 5]], [[1, 3, 6], [4]], [[1, 3, 6], [5]], [[1, 4, 6], [2, 5]], [[1, 4, 6], [3, 5]], [[1, 4, 6], [5]], [[2, 3, 4, 6], [2, 5]], [[2, 3, 4, 6], [3, 5]], [[2, 3, 4, 6], [4, 5]], [[2, 3, 4, 6], [5]], [[2, 3, 5], [4]], [[2, 3, 5], [4, 6]], [[2, 3, 5, 6], [4]], [[2, 3, 5, 6], [4, 5]], [[2, 3, 5, 6], [4, 6]], [[2, 3, 6], [4]], [[2, 3, 6], [5]], [[2, 4, 6], [3, 5]], [[2, 4, 6], [5]], [[3, 4, 6], [5]]]
Rank3ClusterVariablesSL6=[[[1, 2, 3, 4, 6], [1, 2, 4], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 5, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [3]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 6]], [[1, 2, 3, 4, 6], [1, 2, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 6]], [[1, 2, 3, 4, 6], [1, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [4, 5]], [[1, 2, 3, 4, 6], [2, 3, 5], [4]], [[1, 2, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [2, 3, 5, 6], [4, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 5]], [[1, 2, 3, 5], [1, 2, 6], [4]], [[1, 2, 3, 5], [1, 3, 6], [2, 4]], [[1, 2, 3, 5], [1, 3, 6], [4]], [[1, 2, 3, 5], [1, 4, 6], [2, 4]], [[1, 2, 3, 5], [1, 4, 6], [2, 5]], [[1, 2, 3, 5], [1, 4, 6], [3, 4]], [[1, 2, 3, 5], [1, 4, 6], [3, 5]], [[1, 2, 3, 5], [1, 4, 6], [4]], [[1, 2, 3, 5], [1, 4, 6], [5]], [[1, 2, 3, 5], [2, 3, 6], [4]], [[1, 2, 3, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5], [2, 4, 6], [3, 5]], [[1, 2, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5], [2, 4, 6], [5]], [[1, 2, 3, 5], [3, 4, 6], [4]], [[1, 2, 3, 5], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 5], [4]], [[1, 2, 3, 5, 6], [1, 2, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 2, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6]], [[1, 2, 3, 5, 6], [1, 3, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4]], [[1, 2, 3, 5, 6], [1, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4, 6]], [[1, 2, 3, 5, 6], [1, 4, 5], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 6], [5]], [[1, 2, 3, 5, 6], [2, 3, 5], [4]], [[1, 2, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [2, 3, 6], [4]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4, 6]], [[1, 2, 3, 5, 6], [2, 4, 5], [4]], [[1, 2, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [3, 4, 5], [4]], [[1, 2, 3, 5, 6], [3, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [3, 4, 6], [5]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 3, 6], [1, 4], [2, 5]], [[1, 2, 3, 6], [1, 4], [3, 5]], [[1, 2, 3, 6], [1, 4], [5]], [[1, 2, 3, 6], [1, 4, 6], [2, 5]], [[1, 2, 3, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 6], [1, 4, 6], [5]], [[1, 2, 3, 6], [2, 4], [3, 5]], [[1, 2, 3, 6], [2, 4], [5]], [[1, 2, 3, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 6], [2, 4, 6], [5]], [[1, 2, 3, 6], [3, 4], [5]], [[1, 2, 3, 6], [3, 4, 6], [5]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 5]], [[1, 2, 4, 5], [1, 4, 6], [3, 5]], [[1, 2, 4, 5], [2, 4, 6], [3, 5]], [[1, 2, 4, 5], [3, 4, 6], [3, 5]], [[1, 2, 4, 5], [3, 4, 6], [5]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 4, 5, 6], [1, 4, 6], [3, 5]], [[1, 2, 4, 5, 6], [2, 4, 6], [3, 5]], [[1, 2, 4, 5, 6], [3, 4, 6], [3, 5]], [[1, 2, 4, 5, 6], [3, 4, 6], [5]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 4, 6], [1, 2, 5], [3]], [[1, 2, 4, 6], [1, 2, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 5], [3, 6]], [[1, 2, 4, 6], [1, 3, 4], [3, 5]], [[1, 2, 4, 6], [1, 3, 5], [3]], [[1, 2, 4, 6], [1, 3, 5], [3, 5]], [[1, 2, 4, 6], [1, 3, 5], [3, 6]], [[1, 2, 4, 6], [1, 3, 5], [4]], [[1, 2, 4, 6], [1, 3, 5], [4, 5]], [[1, 2, 4, 6], [1, 3, 5], [4, 6]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5]], [[1, 2, 4, 6], [1, 3, 5, 6], [4, 5]], [[1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 4, 6], [1, 4, 6], [3, 5]], [[1, 2, 4, 6], [1, 5, 6], [3, 5]], [[1, 2, 4, 6], [2, 3, 4], [3, 5]], [[1, 2, 4, 6], [2, 3, 5], [3]], [[1, 2, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 4, 6], [2, 3, 5], [3, 6]], [[1, 2, 4, 6], [2, 3, 5], [4]], [[1, 2, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 4, 6], [2, 3, 5], [4, 6]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5]], [[1, 2, 4, 6], [2, 3, 5, 6], [4, 5]], [[1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 4, 6], [2, 5, 6], [3, 5]], [[1, 2, 4, 6], [3], [5]], [[1, 2, 4, 6], [3, 4], [5]], [[1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 4, 6], [3, 4, 6], [5]], [[1, 2, 4, 6], [3, 5], [3]], [[1, 2, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [3, 5], [3, 6]], [[1, 2, 4, 6], [3, 5], [4]], [[1, 2, 4, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [3, 5], [4, 6]], [[1, 2, 4, 6], [3, 5, 6], [5]], [[1, 2, 5], [1, 3, 6], [4]], [[1, 2, 5], [2, 3, 6], [4]], [[1, 2, 5], [3, 6], [4]], [[1, 2, 5, 6], [1, 3, 5], [4]], [[1, 2, 5, 6], [1, 3, 5], [4, 6]], [[1, 2, 5, 6], [1, 3, 6], [4]], [[1, 2, 5, 6], [2, 3, 5], [4]], [[1, 2, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 5, 6], [2, 3, 6], [4]], [[1, 2, 5, 6], [3, 5], [4]], [[1, 2, 5, 6], [3, 5], [4, 6]], [[1, 2, 5, 6], [3, 6], [4]], [[1, 3, 4, 6], [2, 3, 5], [4]], [[1, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 3, 4, 6], [2, 3, 5], [4, 6]], [[1, 3, 4, 6], [2, 3, 5, 6], [4, 5]], [[1, 3, 5], [2, 3, 6], [4]], [[1, 3, 5], [2, 4, 6], [4]], [[1, 3, 5], [2, 4, 6], [5]], [[1, 3, 5, 6], [2, 3, 5], [4]], [[1, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 3, 5, 6], [2, 3, 6], [4]], [[1, 3, 5, 6], [2, 4, 5], [4]], [[1, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 3, 5, 6], [2, 4, 6], [4]], [[1, 3, 5, 6], [2, 4, 6], [5]], [[1, 3, 6], [2, 4], [5]], [[1, 3, 6], [2, 4, 6], [5]]]
Rank4ClusterVariablesSL6=[[[1, 2, 3, 4, 6], [1, 2, 4, 6], [1, 2, 5], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [1, 3, 5], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [1, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [3, 5], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [1, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [1, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5], [1, 3, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [2, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [2, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5], [2, 3, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [1, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [1, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [1, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [2, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 6], [5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5], [4]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 2, 3, 5], [1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4], [5]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 5], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5], [1, 3, 5, 6], [2, 4, 6], [4]], [[1, 2, 3, 5], [1, 3, 6], [2, 4], [4]], [[1, 2, 3, 5], [1, 3, 6], [2, 4], [5]], [[1, 2, 3, 5], [1, 3, 6], [2, 4, 6], [4]], [[1, 2, 3, 5], [1, 3, 6], [2, 4, 6], [5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4], [5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4, 6], [5]], [[1, 2, 3, 5], [1, 4, 6], [3, 4], [5]], [[1, 2, 3, 5], [1, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 5], [2, 4, 6], [3, 4], [5]], [[1, 2, 3, 5], [2, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [1, 4, 6], [2, 4]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [1, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [1, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [1, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [1, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 5], [1, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 5], [1, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 5], [2, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 5], [4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [1, 4, 6], [2, 4]], [[1, 2, 3, 5, 6], [1, 3, 5], [1, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [2, 4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 3, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5, 6], [2, 4, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 5, 6], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4], [4]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4], [5]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4], [5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 4], [5]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [2, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [2, 3, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [2, 3, 5], [4, 6], [4]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 4], [5]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 6], [1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [1, 5], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [2, 5], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 4], [5]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 5], [5]], [[1, 2, 3, 6], [1, 3, 6], [2, 4], [5]], [[1, 2, 3, 6], [1, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 6], [1, 4, 6], [2, 4], [5]], [[1, 2, 3, 6], [1, 4, 6], [2, 5], [3, 5]], [[1, 2, 3, 6], [1, 4, 6], [2, 5], [5]], [[1, 2, 3, 6], [1, 4, 6], [3, 4], [5]], [[1, 2, 3, 6], [1, 4, 6], [3, 5], [5]], [[1, 2, 3, 6], [2, 4, 6], [3, 4], [5]], [[1, 2, 3, 6], [2, 4, 6], [3, 5], [5]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 5, 6], [3, 5]], [[1, 2, 4, 5, 6], [1, 2, 4, 5], [3, 4, 6], [3, 5]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 5, 6], [3, 5]], [[1, 2, 4, 6], [1, 2, 4], [3, 5], [3]], [[1, 2, 4, 6], [1, 2, 4], [3, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 4], [3, 5], [3, 6]], [[1, 2, 4, 6], [1, 2, 4, 6], [1, 3, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [1, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [2, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 5], [5]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 5, 6], [3, 5]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [3]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [3, 6]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [4]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [4, 6]], [[1, 2, 4, 6], [1, 2, 5], [3, 6], [3]], [[1, 2, 4, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [3]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [3, 6]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [4]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [4, 6]], [[1, 2, 4, 6], [1, 3, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 4, 6], [1, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 4, 6], [1, 3, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [1, 3, 4, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [1, 3, 5], [2, 3, 5], [4]], [[1, 2, 4, 6], [1, 3, 5], [2, 3, 5], [4, 6]], [[1, 2, 4, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 4, 6], [1, 3, 5], [3, 5], [4]], [[1, 2, 4, 6], [1, 3, 5], [3, 5], [4, 6]], [[1, 2, 4, 6], [1, 3, 5], [3, 6], [4]], [[1, 2, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4]], [[1, 2, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 5]], [[1, 2, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5], [4]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5], [4, 6]], [[1, 2, 4, 6], [1, 4, 6], [2, 5], [3, 5]], [[1, 2, 4, 6], [1, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [1, 4, 6], [3, 5], [5]], [[1, 2, 4, 6], [2, 3, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [2, 3, 4, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [2, 3, 5], [3, 5], [4]], [[1, 2, 4, 6], [2, 3, 5], [3, 5], [4, 6]], [[1, 2, 4, 6], [2, 3, 5], [3, 6], [4]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5], [4]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5], [4, 6]], [[1, 2, 4, 6], [2, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [2, 4, 6], [3, 5], [5]], [[1, 2, 4, 6], [3, 4, 6], [3, 5], [5]], [[1, 2, 5, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 5, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 5, 6], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 5, 6], [1, 3, 5], [3, 6], [4]], [[1, 2, 5, 6], [1, 3, 5], [4, 6], [4]], [[1, 2, 5, 6], [2, 3, 5], [3, 6], [4]], [[1, 2, 5, 6], [2, 3, 5], [4, 6], [4]], [[1, 3, 5, 6], [1, 3, 5], [2, 4, 6], [4]], [[1, 3, 5, 6], [2, 3, 5], [2, 4, 6], [4]], [[1, 3, 5, 6], [2, 3, 5], [4, 6], [4]]]

In [5]:
# with open('SL6ClusterVariablesRank4', 'w') as fp:
#     for i in Rank4ClusterVariablesSL6:
#         fp.write(str(i)+'\n')

In [3]:
# this part is to compute ch(T) in C[SLk/N] and C[N] case

k=5
T1=[[1,2,4,5],[3,5]] # T1 is a tableau in the case of C[SLk/N] or C[N], T1[i]'s are columns of the tableau
t2=TableauToFundamentalTableauxSLk('Y',T1)
#print(t2)
t3=TableauToListsIJSLk('Y',t2)
#print(t3)
A=t3[0]
B=t3[1]
print(len(A))
Bsorted=sorted(B)
w=LongestPerm(A,Bsorted,B)
w

4


s3*s2

In [4]:
kk=len(A)  
W = SymmetricGroup(kk)
[s1,s2,s3] = W.simple_reflections() # number of s_i is kk-1
P.<q> = LaurentPolynomialRing(QQ)
KL = KazhdanLusztigPolynomial(W,q)

w0=W.long_element() 
l=list(W)

w=s3*s2 # w is computed in previous step, if w=1, then w=W.one()

#t1=[(-1)^(w.length())*KL.P(w0, w*w0)]
t1=[]
for i in l:
    #print(i)
    t2=KL.P(i*w0, w*w0)
    t2=t2.subs({q:1})
    #print(t2)
    t3=(-1)^((i*w).length())
    t1.append(t3*t2)
p1=t1

ll=[]
for i in l:
    ll.append(ShiftByA(i.tuple(),0))

r1=[]
for i in ll:
    t1=SymmetricGroupActionOnTableauxComputeChSLk('Y',k,i,T1)
    r1.append(t1);

r2=[]
for i in range(len(r1)): 
    t1=[p1[i], r1[i]]
    r2.append(t1) 
r2=CollectNonZeros2(r2)
r2=combineLikeTerms3(r2)
r2

[[1, [[1, 2, 3, 5], [1, 2, 4], [1, 5], [3]]],
 [1, [[1, 2, 3, 4], [1, 2, 5], [1, 3], [5]]],
 [1, [[1, 2, 3], [1, 2, 3, 5], [1, 4], [5]]],
 [-1, [[1, 2, 3], [1, 2, 3, 5], [1, 5], [4]]],
 [-1, [[1, 2, 3, 4], [1, 2, 5], [1, 5], [3]]],
 [-1, [[1, 2, 3, 5], [1, 2, 4], [1, 3], [5]]]]

In [8]:
# the following is to compute prime tableaux of rank 1 and rank 2 for C[SL_6/N] and C[N], N \subset SL6, rank is the number of columns of a tableau

k=6
rr=[]
for i in [1..k]:
    t1=[]
    for j in [1..i]:
        t1.append(1)
    p1=SemistandardTableaux(t1, max_entry=k)
    rr=rr+list(p1)
rr1=[]
for i in rr:
    t1=TransposeTableauSLk(i)
    rr1.append(t1)
    
Rank1PrimeSL6=rr1

t1=rr1
tt2=[]
for i1 in t1:
    i=i1
    for j1 in t1:
        j=j1 
        if IsCompatiblePairTableauxInMatrixSLk(k, i, j) and (UnionOfTwoTableauxSLk(i,j) in tt2)==False:
            tt2.append(UnionOfTwoTableauxSLk(i,j))
print(len(tt2))
 
k=6
t1=[]
for i in [0..k-1]:
    for j in [0..i]:
        t2=[]
        for g in [0..j]:
            t2.append(2)
        for g in [j+1..i]:
            t2.append(1)
        t1.append(t2)
    
#print(t1)
rr=[]
for i in t1:
    p1=SemistandardTableaux(i, max_entry=k)
    rr=rr+list(p1)

rr1=[]
for i in rr: 
    t1=TransposeTableauSLk(i)
    rr1.append(t1)
len(rr1)

rr2=SetDifference(rr1,tt2)
Rank2PrimeSL6=rr2
len(rr2)

1533


119

In [ ]:
# prime tableaux of rank 3
t1=Rank1PrimeSL6
tt2=[]
for i1 in t1:
    i=i1
    for j1 in Rank2PrimeSL6: # prime tableaux of rank 2
        j=j1
        if IsCompatiblePairTableauxInMatrixSLk(k, i, j)==1:
            tt2.append(UnionOfTwoTableauxSLk(i,j))
print(len(tt2))
 
t1=Rank1PrimeSL6
tt3=[]
for i1 in t1:
    i=i1
    for j1 in t1: 
        j=j1
        for c1 in t1:
            c=c1
            if IsWeaklySeparatedSLk(i[0],j[0])==1 and IsWeaklySeparatedSLk(i[0],c[0])==1 and IsWeaklySeparatedSLk(c[0],j[0])==1 and (UnionOfTableauxManySLk([i,j,c]) in tt3)==False:
                tt3.append(UnionOfTableauxManySLk([i,j,c]))
len(tt3)

k=6 
t1=[]
for i in [0..k-1]:
    for j in [0..i]:
        for j1 in [0..j]:
            t2=[]
            for g in [0..j1]:
                t2.append(3)
            for g in [j1+1..j]:
                t2.append(2)
            for g in [j+1..i]:
                t2.append(1)
            t1.append(t2)
    
print(t1)
rr=[]
for i in t1:
    p1=SemistandardTableaux(i, max_entry=k)
    rr=rr+list(p1)
len(rr)
rank3tableauxSL6=[]
for i in rr:
    t1=TransposeTableauSLk(i)
    rank3tableauxSL6.append(t1)
len(rank3tableauxSL6)

Rank3PrimeSL6=SetDifference(SetDifference(rank3tableauxSL6, tt2), tt3)
len(Rank3PrimeSL6)

In [ ]:
25740-21657-3913

In [ ]:
# prime tableaux of rank 4, takes very long time
t2=[]
for i1 in Rank1PrimeSL6:
    for i2 in Rank1PrimeSL6:
        for j1 in Rank2PrimeSL6: # prime tableaux of rank 2
            if IsCompatiblePairTableauxInMatrixSLk(k, i1, j1)==1 and IsCompatiblePairTableauxInMatrixSLk(k, i2, j1)==1 and IsWeaklySeparatedSLk(i1[0],i2[0])==1 and (UnionOfTableauxManySLk([i1,i2,j1]) in t2)==False:
                t2.append(UnionOfTableauxManySLk([i1,i2,j1]))
print(len(t2))
 
t1=Rank1PrimeSL6
t3=[]
for i1 in t1: 
    for j1 in t1: 
        for c1 in t1: 
            for d1 in t1: 
                if IsWeaklySeparatedSLk(i1[0],j1[0])==1 and IsWeaklySeparatedSLk(i1[0],c1[0])==1 and IsWeaklySeparatedSLk(i1[0],d1[0])==1 and IsWeaklySeparatedSLk(j1[0],c1[0])==1 and IsWeaklySeparatedSLk(j1[0],d1[0])==1 and IsWeaklySeparatedSLk(c1[0],d1[0])==1 and (UnionOfTableauxManySLk([i1,j1,c1,d1]) in t3)==False:
                    t3.append(UnionOfTableauxManySLk([i1,j1,c1,d1]))
len(t3)

t4=[]
for i1 in Rank1PrimeSL6: 
    for j1 in Rank3PrimeSL6: # prime tableaux of rank 3
        if IsCompatiblePairTableauxInMatrixSLk(k, i1, j1)==1 and (UnionOfTableauxManySLk([i1,j1]) in t4)==False:
            t4.append(UnionOfTableauxManySLk([i1,j1]))
print(len(t4))

k=6   
r5=[]
for i1 in Rank2PrimeSL6: 
    for j1 in Rank2PrimeSL6: # prime tableaux of rank 2
        if IsCompatiblePairTableauxInMatrixSLkUsingChT(k, i1, j1)==1:  # this part needs to use Gregory S. Warrington's C program of computing KL polynomials: https://gswarrin.w3.uvm.edu/research/klc/klc.html, download the program and put all the files in the same folder as this SageMath file, and use gcc to compile the C codes
            t5.append(UnionOfTableauxManySLk([i1,j1]))
r5=removeDuplicatesListOfLists(r5)
print(len(r5))

k=6 
t1=[]
for i in [0..k-1]:
    for j in [0..i]:
        for j1 in [0..j]:
            for j2 in [0..j1]:
                p2=[]
                for g1 in [0..j2]:
                    p2.append(4)
                for g in [j2+1..j1]:
                    p2.append(3)
                for g in [j1+1..j]:
                    p2.append(2)
                for g in [j+1..i]:
                    p2.append(1)
                t1.append(p2)
    
#print(t1)
rr=[]
for i in t1:
    p1=SemistandardTableaux(i, max_entry=k)
    rr=rr+list(p1)
len(rr)
rank4tableauxSL6=[]
for i in rr:
    t1=TransposeTableauSLk(i)
    rank4tableauxSL6.append(t1)
print(len(rank4tableauxSL6))

Rank4PrimeSL6=SetDifference(SetDifference(SetDifference(SetDifference(rank4tableauxSL6, t2), t3), t4), t5)
len(Rank4PrimeSL6)

In [ ]:
279279-212127-60966-4322-1649

In [2]:
# this is the result of prime tableaux for C[SL_5/N] that I obtained by the above computations
# rank 2 case has a non-real tableau, rank 1 case includes frozen variables
Rank1PrimeSL6=[[[1]], [[2]], [[3]], [[4]], [[5]], [[6]], [[1, 2]], [[1, 3]], [[1, 4]], [[1, 5]], [[1, 6]], [[2, 3]], [[2, 4]], [[2, 5]], [[2, 6]], [[3, 4]], [[3, 5]], [[3, 6]], [[4, 5]], [[4, 6]], [[5, 6]], [[1, 2, 3]], [[1, 2, 4]], [[1, 2, 5]], [[1, 2, 6]], [[1, 3, 4]], [[1, 3, 5]], [[1, 3, 6]], [[1, 4, 5]], [[1, 4, 6]], [[1, 5, 6]], [[2, 3, 4]], [[2, 3, 5]], [[2, 3, 6]], [[2, 4, 5]], [[2, 4, 6]], [[2, 5, 6]], [[3, 4, 5]], [[3, 4, 6]], [[3, 5, 6]], [[4, 5, 6]], [[1, 2, 3, 4]], [[1, 2, 3, 5]], [[1, 2, 3, 6]], [[1, 2, 4, 5]], [[1, 2, 4, 6]], [[1, 2, 5, 6]], [[1, 3, 4, 5]], [[1, 3, 4, 6]], [[1, 3, 5, 6]], [[1, 4, 5, 6]], [[2, 3, 4, 5]], [[2, 3, 4, 6]], [[2, 3, 5, 6]], [[2, 4, 5, 6]], [[3, 4, 5, 6]], [[1, 2, 3, 4, 5]], [[1, 2, 3, 4, 6]], [[1, 2, 3, 5, 6]], [[1, 2, 4, 5, 6]], [[1, 3, 4, 5, 6]], [[2, 3, 4, 5, 6]], [[1, 2, 3, 4, 5, 6]]]
Rank2PrimeSL6=[[[1, 3, 5], [2, 4, 6]], [[1, 2, 4], [3, 5, 6]], [[1, 3, 5], [2, 4]], [[1, 2, 4], [3, 5]], [[1, 3, 6], [2, 4]], [[1, 2, 4], [3, 6]], [[1, 3, 6], [2, 5]], [[1, 2, 5], [3, 6]], [[1, 4, 6], [2, 5]], [[1, 2, 5], [4, 6]], [[1, 4, 6], [3, 5]], [[1, 3, 5], [4, 6]], [[2, 4, 6], [3, 5]], [[2, 3, 5], [4, 6]], [[1, 2, 4], [3]], [[1, 2, 5], [3]], [[1, 2, 6], [3]], [[1, 2, 5], [4]], [[1, 2, 6], [4]], [[1, 2, 6], [5]], [[1, 3, 5], [4]], [[1, 3, 6], [4]], [[1, 3, 6], [5]], [[1, 4, 6], [5]], [[2, 3, 5], [4]], [[2, 3, 6], [4]], [[2, 3, 6], [5]], [[2, 4, 6], [5]], [[3, 4, 6], [5]], [[1, 2, 4, 6], [1, 3, 5]], [[1, 2, 3, 5], [1, 4, 6]], [[1, 2, 4, 6], [2, 3, 5]], [[1, 2, 3, 5], [2, 4, 6]], [[1, 3, 4, 6], [2, 3, 5]], [[1, 2, 3, 5], [3, 4, 6]], [[1, 3, 4, 6], [2, 4, 5]], [[1, 2, 4, 5], [3, 4, 6]], [[1, 3, 5, 6], [2, 4, 5]], [[1, 2, 4, 5], [3, 5, 6]], [[1, 3, 5, 6], [2, 4, 6]], [[1, 2, 4, 6], [3, 5, 6]], [[1, 2, 3, 5], [1, 4]], [[1, 2, 3, 6], [1, 4]], [[1, 2, 3, 6], [1, 5]], [[1, 2, 4, 6], [1, 5]], [[1, 3, 4, 6], [1, 5]], [[1, 2, 3, 5], [2, 4]], [[1, 2, 3, 6], [2, 4]], [[1, 2, 3, 6], [2, 5]], [[1, 2, 4, 6], [2, 5]], [[1, 2, 3, 5], [3, 4]], [[1, 2, 3, 6], [3, 4]], [[1, 2, 3, 6], [3, 5]], [[1, 2, 4, 5], [3, 4]], [[1, 2, 4, 6], [3, 4]], [[1, 2, 4, 5], [3, 5]], [[1, 3, 5, 6], [2, 4]], [[1, 3, 4, 6], [2, 5]], [[1, 2, 4, 6], [3, 5]], [[1, 2, 4, 5], [3, 6]], [[1, 2, 3, 6], [4, 5]], [[1, 2, 3, 5], [4, 6]], [[1, 2, 4, 6], [3, 6]], [[1, 2, 5, 6], [3, 5]], [[1, 2, 5, 6], [3, 6]], [[1, 2, 4, 6], [4, 5]], [[1, 2, 5, 6], [4, 5]], [[1, 2, 5, 6], [4, 6]], [[1, 3, 4, 6], [3, 5]], [[1, 3, 4, 6], [4, 5]], [[1, 3, 5, 6], [4, 5]], [[1, 3, 5, 6], [4, 6]], [[2, 3, 4, 6], [2, 5]], [[2, 3, 4, 6], [3, 5]], [[2, 3, 4, 6], [4, 5]], [[2, 3, 5, 6], [4, 5]], [[2, 3, 5, 6], [4, 6]], [[1, 2, 4, 5], [3]], [[1, 2, 3, 5], [4]], [[1, 2, 4, 6], [3]], [[1, 2, 3, 6], [4]], [[1, 2, 5, 6], [3]], [[1, 2, 3, 6], [5]], [[1, 2, 5, 6], [4]], [[1, 2, 4, 6], [5]], [[1, 3, 5, 6], [4]], [[1, 3, 4, 6], [5]], [[2, 3, 5, 6], [4]], [[2, 3, 4, 6], [5]], [[1, 2, 3, 4, 6], [1, 2, 5]], [[1, 2, 3, 4, 6], [1, 3, 5]], [[1, 2, 3, 4, 6], [1, 4, 5]], [[1, 2, 3, 5, 6], [1, 4, 5]], [[1, 2, 3, 5, 6], [1, 4, 6]], [[1, 2, 3, 4, 6], [2, 3, 5]], [[1, 2, 3, 4, 6], [2, 4, 5]], [[1, 2, 3, 5, 6], [2, 4, 5]], [[1, 2, 3, 5, 6], [2, 4, 6]], [[1, 2, 3, 4, 6], [3, 4, 5]], [[1, 2, 3, 5, 6], [3, 4, 5]], [[1, 2, 3, 5, 6], [3, 4, 6]], [[1, 2, 4, 5, 6], [3, 4, 5]], [[1, 2, 4, 5, 6], [3, 4, 6]], [[1, 2, 4, 5, 6], [3, 5, 6]], [[1, 2, 3, 5, 6], [1, 4]], [[1, 2, 3, 4, 6], [1, 5]], [[1, 2, 3, 5, 6], [2, 4]], [[1, 2, 3, 4, 6], [2, 5]], [[1, 2, 3, 5, 6], [3, 4]], [[1, 2, 3, 4, 6], [3, 5]], [[1, 2, 4, 5, 6], [3, 4]], [[1, 2, 3, 4, 6], [4, 5]], [[1, 2, 4, 5, 6], [3, 5]], [[1, 2, 3, 5, 6], [4, 5]], [[1, 2, 4, 5, 6], [3, 6]], [[1, 2, 3, 5, 6], [4, 6]], [[1, 2, 4, 5, 6], [3]], [[1, 2, 3, 5, 6], [4]], [[1, 2, 3, 4, 6], [5]]]
Rank3PrimeSL6=[[[1, 2, 3, 4, 6], [1, 2, 5, 6], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 4, 5, 6], [3, 5]], [[1, 2, 4, 6], [1, 2, 5], [3]], [[1, 2, 3, 4, 6], [1, 2, 4], [3, 5]], [[1, 2, 4, 6], [1, 2, 5], [3, 5]], [[1, 2, 4, 6], [2, 3, 4], [3, 5]], [[1, 2, 4, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [3, 5], [4, 6]], [[1, 2, 5, 6], [3, 5], [4, 6]], [[1, 2, 5, 6], [3, 5], [4]], [[1, 2, 4, 5, 6], [3, 4, 6], [3, 5]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 5]], [[1, 2, 4, 5], [3, 4, 6], [3, 5]], [[1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 4, 6], [3], [5]], [[1, 2, 3, 5], [3, 4, 6], [5]], [[1, 2, 3, 6], [3, 4, 6], [5]], [[1, 2, 3, 5], [1, 4, 6], [3, 5]], [[1, 2, 3, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6]], [[1, 2, 3, 5], [1, 2, 6], [4]], [[1, 2, 3, 5, 6], [3, 4, 5], [4]], [[1, 2, 3, 5], [1, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [4]], [[1, 3, 5, 6], [2, 4, 5], [4]], [[1, 3, 5, 6], [2, 4, 6], [4]], [[1, 2, 3, 5], [1, 4, 6], [4]], [[1, 3, 5, 6], [2, 4, 6], [5]], [[1, 2, 3, 4, 6], [2, 3, 5], [4]], [[1, 2, 3, 5, 6], [2, 3, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4]], [[1, 2, 3, 5, 6], [2, 3, 6], [4]], [[1, 2, 3, 6], [1, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 2, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [4]], [[1, 3, 5], [2, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [3, 4, 5], [4, 6]], [[1, 2, 4, 6], [1, 2, 5], [3, 6]], [[1, 2, 4, 6], [3, 5], [4]], [[1, 2, 5, 6], [3, 6], [4]], [[1, 2, 4, 6], [1, 3, 5], [3]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 3, 6], [1, 4], [3, 5]], [[1, 2, 3, 6], [3, 4], [5]], [[1, 2, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 6]], [[1, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 2, 5, 6], [2, 3, 6], [4]], [[1, 3, 5, 6], [2, 3, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4, 6]], [[1, 2, 3, 5], [3, 4, 6], [4]], [[1, 2, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4, 6]], [[1, 2, 4, 6], [1, 5, 6], [3, 5]], [[1, 2, 4, 6], [2, 5, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 6], [1, 4], [5]], [[1, 2, 3, 6], [2, 4], [5]], [[1, 2, 4, 6], [3, 4, 6], [5]], [[1, 2, 4, 5], [3, 4, 6], [5]], [[1, 2, 3, 5], [1, 4, 6], [2, 5]], [[1, 2, 3, 5], [2, 4, 6], [5]], [[1, 3, 5], [2, 4, 6], [4]], [[1, 3, 5], [2, 4, 6], [5]], [[1, 2, 3, 5], [1, 4, 6], [5]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 4, 6], [3, 4], [5]], [[1, 2, 4, 6], [3, 5, 6], [5]], [[1, 2, 4, 6], [2, 3, 5], [3]], [[1, 3, 4, 6], [2, 3, 5, 6], [4, 5]], [[1, 2, 4, 6], [3, 5], [3]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 5]], [[1, 2, 3, 6], [2, 4, 6], [5]], [[1, 2, 3, 4, 6], [2, 3, 5, 6], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [4, 5]], [[1, 2, 3, 5, 6], [1, 2, 5], [4, 6]], [[1, 2, 4, 6], [2, 3, 5, 6], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [3]], [[1, 2, 3, 5, 6], [3, 4, 6], [5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4]], [[1, 2, 3, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5], [1, 4, 6], [3, 4]], [[1, 2, 4, 6], [1, 3, 5], [3, 6]], [[1, 2, 4, 6], [1, 3, 4], [3, 5]], [[1, 2, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [4]], [[1, 2, 3, 5, 6], [1, 2, 6], [4]], [[1, 2, 5], [2, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6]], [[1, 2, 4, 5, 6], [3, 4, 6], [5]], [[1, 2, 4, 5, 6], [1, 4, 6], [3, 5]], [[1, 2, 4, 6], [1, 4, 6], [3, 5]], [[1, 3, 4, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 6], [2, 4], [3, 5]], [[1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5], [2, 3, 6], [4]], [[1, 2, 4, 5], [1, 4, 6], [3, 5]], [[1, 3, 6], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 5]], [[1, 2, 4, 6], [2, 3, 5], [3, 6]], [[1, 2, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 5], [3, 6], [4]], [[1, 2, 4, 6], [1, 3, 5], [4]], [[1, 2, 3, 4, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4]], [[1, 2, 3, 5, 6], [2, 4, 5], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4]], [[1, 2, 4, 6], [1, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 5]], [[1, 2, 3, 4, 6], [1, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [4, 6]], [[1, 2, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 4]], [[1, 2, 4, 5, 6], [2, 4, 6], [3, 5]], [[1, 2, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 5, 6], [2, 3, 5], [4]], [[1, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 5, 6], [1, 3, 5], [4, 6]], [[1, 2, 5], [1, 3, 6], [4]], [[1, 2, 4, 6], [2, 3, 5], [4]], [[1, 3, 5, 6], [2, 3, 5], [4]], [[1, 3, 4, 6], [2, 3, 5], [4]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5]], [[1, 2, 4, 5], [2, 4, 6], [3, 5]], [[1, 2, 3, 6], [1, 4], [2, 5]], [[1, 2, 4, 6], [2, 3, 5], [4, 6]], [[1, 2, 4, 6], [1, 3, 5], [4, 5]], [[1, 3, 6], [2, 4], [5]], [[1, 2, 4, 6], [1, 3, 5, 6], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 6]], [[1, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5]], [[1, 2, 3, 5], [1, 3, 6], [2, 4]], [[1, 2, 3, 6], [1, 4, 6], [2, 5]], [[1, 2, 5, 6], [1, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4]], [[1, 2, 5, 6], [1, 3, 5], [4]], [[1, 2, 3, 5, 6], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5]], [[1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 4, 6], [1, 3, 5], [3, 5]], [[1, 2, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [3, 5], [3, 6]]]
Rank4PrimeSL6=[[[1, 2, 3, 4, 6], [1, 2, 5, 6], [2, 3, 5], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [1, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [1, 4, 6], [2, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4, 6], [5]], [[1, 2, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 5]], [[1, 2, 5, 6], [1, 3, 5], [3, 6], [4]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [1, 3, 5], [3, 5]], [[1, 2, 3, 5, 6], [1, 3, 5, 6], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5], [4, 6]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4], [5]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 4, 6], [1, 3, 5], [3, 5], [4, 6]], [[1, 2, 4, 6], [2, 3, 5], [3, 6], [4]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [3, 6]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5], [4, 5]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [5]], [[1, 3, 5, 6], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [1, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 4, 6], [5]], [[1, 2, 5, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 4, 6], [1, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 2, 5], [2, 3, 5], [4, 6]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 3, 5, 6], [1, 3, 5], [4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [1, 3, 5], [4, 5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [2, 3, 5], [4, 5]], [[1, 2, 4, 6], [1, 2, 4], [3, 5], [3, 6]], [[1, 2, 3, 6], [1, 4, 6], [3, 5], [5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5], [4, 6]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 3, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4], [4]], [[1, 2, 5, 6], [2, 3, 5], [3, 6], [4]], [[1, 2, 3, 5], [1, 4, 6], [2, 4, 6], [5]], [[1, 2, 4, 5, 6], [1, 2, 4, 5], [3, 4, 6], [3, 5]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [4]], [[1, 2, 3, 5], [1, 3, 6], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [3, 5], [3, 5]], [[1, 2, 3, 5], [1, 4, 6], [2, 4], [5]], [[1, 2, 3, 5], [1, 3, 6], [2, 4], [4]], [[1, 2, 4, 6], [1, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [1, 3, 4, 6], [3, 5], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 4], [5]], [[1, 2, 4, 6], [2, 3, 5], [3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [3, 4, 6], [4]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 4], [5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [1, 3, 5], [4]], [[1, 3, 5, 6], [2, 3, 5], [2, 4, 6], [4]], [[1, 2, 5, 6], [1, 3, 5], [4, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4], [5]], [[1, 2, 4, 6], [1, 4, 6], [3, 5], [5]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5], [4, 6]], [[1, 2, 3, 5], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [3, 6]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 3, 6], [1, 2, 4, 6], [1, 5], [3, 5]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5], [4]], [[1, 2, 3, 6], [2, 4, 6], [3, 5], [5]], [[1, 2, 3, 6], [1, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4, 6], [5]], [[1, 2, 4, 6], [1, 3, 5], [2, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5], [1, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [1, 4, 6], [3, 4]], [[1, 2, 4, 6], [1, 3, 5], [3, 5], [4]], [[1, 2, 4, 6], [2, 3, 4, 6], [3, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [3]], [[1, 2, 4, 6], [2, 4, 6], [3, 5], [3, 5]], [[1, 2, 3, 6], [1, 4, 6], [2, 4], [5]], [[1, 2, 3, 4, 6], [1, 2, 5], [2, 3, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [1, 3, 5], [4, 5]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5], [4]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [4, 5]], [[1, 2, 5, 6], [2, 3, 5], [4, 6], [4]], [[1, 2, 3, 5], [1, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 4, 6], [1, 2, 4, 6], [1, 5], [3, 5]], [[1, 2, 4, 6], [2, 4, 6], [3, 5], [5]], [[1, 2, 3, 5], [1, 4, 6], [3, 4], [5]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5], [4, 6]], [[1, 2, 4, 6], [1, 2, 4], [3, 5], [3]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [2, 4, 5], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 5], [2, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [4]], [[1, 2, 4, 6], [1, 3, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 3, 5], [1, 3, 5, 6], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 2, 5], [4, 6], [4]], [[1, 2, 4, 6], [2, 3, 5], [3, 5], [4]], [[1, 2, 3, 6], [1, 2, 4, 6], [2, 5], [3, 5]], [[1, 2, 3, 4, 6], [1, 3, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 2, 5], [3, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [1, 4, 6], [3, 5]], [[1, 2, 3, 5], [2, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 5], [1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 4, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5], [4]], [[1, 2, 4, 6], [3, 4, 6], [3, 5], [5]], [[1, 2, 3, 5], [2, 4, 6], [3, 4], [5]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [3]], [[1, 2, 3, 6], [1, 4, 6], [3, 4], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4], [5]], [[1, 2, 3, 4, 6], [1, 2, 5], [1, 3, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5, 6], [2, 4, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [1, 4, 6], [4]], [[1, 2, 4, 6], [1, 3, 5], [2, 3, 5], [4]], [[1, 2, 4, 6], [1, 2, 4, 6], [2, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 4], [3, 5], [3, 5]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4]], [[1, 2, 3, 5, 6], [1, 4, 6], [3, 4], [5]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [1, 4, 6], [4]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 5, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 5, 6], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 5], [1, 3, 6], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4], [4, 6]], [[1, 2, 4, 5], [1, 2, 4, 6], [3, 5, 6], [3, 5]], [[1, 2, 3, 5, 6], [2, 4, 6], [3, 4, 6], [5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5], [4, 6]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [4, 6]], [[1, 2, 3, 5, 6], [2, 3, 5], [4, 6], [4]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 5], [4, 6]], [[1, 2, 3, 5, 6], [1, 3, 6], [2, 4, 6], [5]], [[1, 2, 3, 6], [1, 2, 4, 6], [1, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 5], [3, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 6], [4]], [[1, 2, 3, 6], [2, 4, 6], [3, 4], [5]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 3, 5], [4, 6]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [1, 2, 5], [3, 5]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [2, 4]], [[1, 2, 3, 4, 6], [1, 2, 5], [3, 5], [4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [2, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [2, 3, 5], [3, 4, 6], [4]], [[1, 2, 4, 6], [1, 3, 5, 6], [3, 5], [4, 5]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [2, 4, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [2, 3, 5], [4, 6]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 5], [2, 4, 6], [4]], [[1, 2, 3, 5], [1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4, 6], [4]], [[1, 2, 3, 6], [1, 2, 4, 6], [3, 5], [5]], [[1, 2, 3, 4, 6], [1, 2, 5], [2, 3, 6], [4]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 6], [5]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [2, 3, 5], [3, 5]], [[1, 2, 4, 6], [1, 3, 5, 6], [2, 3, 5], [4]], [[1, 3, 5, 6], [2, 3, 5], [4, 6], [4]], [[1, 2, 3, 5], [1, 3, 6], [2, 4, 6], [5]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 5, 6], [3, 5]], [[1, 2, 3, 5], [1, 3, 6], [2, 4], [5]], [[1, 2, 4, 6], [1, 3, 4, 6], [3, 5], [4, 5]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 5], [3, 4, 6], [5]], [[1, 2, 3, 6], [1, 4, 6], [2, 5], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 3, 5], [2, 4, 6], [3, 4]], [[1, 2, 3, 5, 6], [1, 4, 6], [2, 4], [5]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [3, 4, 6], [3, 4]], [[1, 2, 3, 4, 6], [1, 2, 4, 6], [2, 3, 5], [4, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [1, 3, 5], [3, 5]], [[1, 2, 4, 6], [1, 2, 4, 6], [3, 5], [5]], [[1, 2, 4, 6], [1, 2, 5, 6], [3, 5], [4, 6]], [[1, 2, 4, 6], [2, 3, 5, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [1, 3, 5], [3, 6], [4]], [[1, 2, 4, 6], [1, 2, 5], [3, 6], [3]], [[1, 2, 3, 6], [1, 4, 6], [2, 5], [5]], [[1, 2, 3, 6], [1, 2, 4, 6], [2, 4], [3, 5]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4], [4]], [[1, 2, 3, 4, 6], [1, 3, 5, 6], [2, 4, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 5], [4]], [[1, 2, 3, 5, 6], [1, 3, 5], [1, 4, 6], [2, 4]], [[1, 2, 3, 5, 6], [1, 2, 4, 5], [1, 4, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 3, 5], [2, 3, 5], [4]], [[1, 2, 3, 5, 6], [2, 3, 5], [2, 4, 6], [4]], [[1, 2, 4, 6], [1, 3, 5], [2, 3, 6], [4]], [[1, 2, 4, 6], [2, 3, 4, 6], [3, 5], [4, 5]], [[1, 2, 4, 6], [1, 2, 5], [3, 5], [4]], [[1, 2, 4, 6], [1, 4, 6], [2, 5], [3, 5]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4], [3, 5]], [[1, 2, 4, 5, 6], [1, 2, 4, 6], [3, 5, 6], [3, 5]], [[1, 2, 3, 4, 6], [1, 2, 5, 6], [1, 3, 5], [4, 6]], [[1, 2, 3, 6], [1, 3, 6], [2, 4], [5]], [[1, 2, 3, 5, 6], [1, 2, 5], [1, 4, 6], [4]], [[1, 2, 3, 5], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 2, 4, 6], [3, 4, 6], [3, 5]], [[1, 2, 3, 5, 6], [1, 4, 5], [2, 4, 6], [4]]]


In [ ]:
# F1 = open('Rank4PrimeSL6','w')
# for i in Rank4PrimeSL6:
#     F1.write(str(i)+'\n')
# F1.close()

In [ ]:
t1=SetDifference(Rank4PrimeSL6, Rank4ClusterVariablesSL6)
len(t1)

In [7]:
# The following is a faster way to get the above result
# instead of using 
# k=6   
# r5=[]
# for i1 in Rank2PrimeSL6: 
#     for j1 in Rank2PrimeSL6: # prime tableaux of rank 2
#         if IsCompatiblePairTableauxInMatrixSLkUsingChT(k, i1, j1)==1:  # this part needs to use Gregory S. Warrington's C program of computing KL polynomials: https://gswarrin.w3.uvm.edu/research/klc/klc.html, download the program and put all the files in the same folder as this SageMath file, and use gcc to compile the C codes
#             t5.append(UnionOfTableauxManySLk([i1,j1]))
# r5=removeDuplicatesListOfLists(r5)
# print(len(r5))
# we use
k=6
t5=[]
for i1 in Rank2PrimeSL6: 
    for j1 in Rank2PrimeSL6: # prime tableaux of rank 2
        if IsCompatiblePairTableauxInMatrixSLk(k, i1, j1)==1:
            t5.append(UnionOfTableauxManySLk([i1,j1]))
t5=removeDuplicatesListOfLists(t5)
print(len(t5))
# but this set t5 has 5 elements in common with the set t2 above, this is because LC is only necessary condition
# on the other hand, removing from rank4tableauxSL6 the sets t2, t3, t4, t5, we still get correct set of prime tableaux for SL6 of rank 4 which has 215 elements

1654


In [3]:
non_real_2times2=[[1,2,3,5,6],[1,3,5],[2,4,6],[4]]
ee1=[]
for i1 in Rank2PrimeSL6: 
    for j1 in Rank2PrimeSL6: 
        if UnionOfTableauxManySLk([i1,j1])==non_real_2times2:
            print(i1,j1)    # need to check if these pairs compatible using IsCompatiblePairTableauxInMatrixSLkUsingChT
            ee1.append(sorted([i1,j1]))
ee1=removeDuplicatesListOfLists(ee1)
len(ee1)

[[1, 3, 5], [2, 4, 6]] [[1, 2, 3, 5, 6], [4]]
[[1, 3, 5], [4]] [[1, 2, 3, 5, 6], [2, 4, 6]]
[[1, 3, 6], [4]] [[1, 2, 3, 5, 6], [2, 4, 5]]
[[2, 3, 5], [4]] [[1, 2, 3, 5, 6], [1, 4, 6]]
[[2, 3, 6], [4]] [[1, 2, 3, 5, 6], [1, 4, 5]]
[[1, 2, 3, 5, 6], [1, 4, 5]] [[2, 3, 6], [4]]
[[1, 2, 3, 5, 6], [1, 4, 6]] [[2, 3, 5], [4]]
[[1, 2, 3, 5, 6], [2, 4, 5]] [[1, 3, 6], [4]]
[[1, 2, 3, 5, 6], [2, 4, 6]] [[1, 3, 5], [4]]
[[1, 2, 3, 5, 6], [4]] [[1, 3, 5], [2, 4, 6]]


5

In [6]:
k=6
i=ee1[0]
IsCompatiblePairTableauxInMatrixSLkUsingChT(k, i[0],i[1])

4
w (2,3)
4
w (1,2,3)
[1, [[1, 2, 3, 5, 6], [1, 3, 5], [2, 4, 6], [4]]]
8
w (1,2,4)(5,7,8)
[-1, [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5], [6]]]
7
w (1,2,4,6,7)
[1, [[1, 2, 3, 5, 6], [4, 5, 6], [4]]]
6
w (1,2)(3,5,6,4)
[1, [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 6], [5]]]
7
w (1,2,4,6,7,5)
[1, [[1, 2, 3, 4, 6], [1, 3, 5], [2, 4, 5], [6]]]
7
w (1,2,4,6,7)
[1, [[1, 2, 3, 4, 6], [1, 4, 5], [5], [6]]]
5
w (1,3,2,4,5)
[1, [[1, 2, 3, 4, 6], [4, 5, 6], [5]]]
5
w (1,2,3)(4,5)
[-1, [[1, 2, 3, 4, 6], [1, 4, 5], [5], [6]]]
5
w (1,3,2,4,5)
[1, [[1, 4, 5], [6], [6]]]
4
w (1,3,2,4)
[1, [[4, 5, 6], [6]]]
4
w (1,2,3,4)
[-1, [[1, 4, 5], [6], [6]]]
4
w (1,3,2,4)


0